## Atom Builder.
Atom Builder renders a 3D charge field model of the atom (1-90) 
selected by the user. 

Atom Builder is intended to introduce Charge field theory. It's 
also my first Jupyter Notebook project and you may notice my 
understanding and coding skills have forced me to alter and 
simplify a few things a bit. Please pardon my errors. Feel free 
to help make this project better.

The only true particles in nature are photons, real spinning 
objects with mass and radii as small as 10^(-27)m. Photons travel 
with both forward and angular momentum (e=mc^2) due to both the 
photon's forward and spin tangential light speed velocities. Real 
spinning photons create both electric (due to the photon's linear 
momentum) and magnetic (due to the photon's angular momentum) 
fields. All spinning matter larger than photons such as electrons,
protons, planets, suns or galaxies are comprised of and constantly 
recycle photons, creating the charge field.

Charge recycling. All protons are constantly receiving and emitting 
photons. Photons can enter a proton from any direction but usually 
do so at the proton's spin axis poles. Protons can emit photons 
outward in any direction but are usually emitted from the higher 
spin angular momentum latitudes about the proton's equator. 
                        
Two protons can "bond" in one of two ways, pole-to-pole, as in the 
Helium atom, (also called an alpha) or equator-to-pole, a charge flow 
male to female connection. An electron or two will be caught in the 
streams of charge entering the proton and must orbit the proton pole, 
like "circling a drain". Neutrons too are caught in the proton charge 
stream and also orbit the proton's pole and spin axis, away from the 
proton's high emission equatorial zone.

Run each cell in turn. The 12th cell is a brief markdown 
explanation and the next cell displays the periodic table. 
After that, the main gui widget controls is displayed. 

In [83]:
from pythreejs import *
from IPython.display import display
from ipywidgets import Button, ButtonStyle, GridspecLayout
import ipywidgets as widgets
from math import sqrt, tan
from random import random , randint , seed

phi = (1+sqrt(5))/2  # calculating phi-based-pi, piG.
piG = 4/sqrt(phi)    # piG is used instead of pi.
pRadius = 0.05 # proton radius, used by all atomic distances
caroSecs = 5 # Problem Here # time in secs for the carousel to make one rotation

In [84]:
view_width = 600
view_height = 400
camera = PerspectiveCamera(position=[5, 0, 0], up=[0, 0, 1], aspect=view_width/view_height)
#camera = PerspectiveCamera(position=[5, 0, 0], up=[1, 0, 0], aspect=view_width/view_height)
camera = PerspectiveCamera(position=[5, 0, 0], up=[0, 1, 0], aspect=view_width/view_height)
key_light = DirectionalLight(position=[0, 5, 5])
ambient_light = AmbientLight()

In [85]:
elements = ((' 1 Hydrogen H ',1),(' 2 Helium He ',2),(' 3 Lithium Li ',3),(' 4 Beryllium Be ',4),
            (' 5 Boron B ',5),(' 6 Carbon C ',6),(' 7 Nitrogen N ',7),(' 8 Oxygen O ',8),
            (' 9 Fluorine F ',9),(' 10 Neon Ne ',10),(' 11 Sodium Na ',11),(' 12 Magnesium Mg ',12),
            (' 13 Aluminum Al ',13),(' 14 Silicon Si ',14),(' 15 Phosphorus P ',15),(' 16 Sulfur S ',16),
            (' 17 Chlorine Cl ',17),(' 18 Argon Ar ',18),(' 19 Potassium P ',19),(' 20 Calcium Ca ',20),
            (' 21 Scandium Sc ',21),(' 22 Titanium Ti ',22),(' 23 Vanadium V ',23),(' 24 Chromium Cr ',24),
            (' 25 Manganese Mn ',25),(' 26 Iron Fe ',26),(' 27 Cobalt Co ',27),(' 28 Nickel Ni ',28),
            (' 29 Copper Cu ',29),(' 30 Zinc Zn ',30),(' 31 Gallium Ga ',31),(' 32 Germanium Ge ',32),
            (' 33 Arsenic As ',33),(' 34 Selenium Se ',34),(' 35 Bromine Br ',35),(' 36 Krypton Kr ',36),
            (' 37 Rubidium Rb ',37),(' 38 Strontium Sr ',38),(' 39 Yttrium Y ',39),(' 40 Zirconium Zr ',40),
            (' 41 Niobium Nb ',41),(' 42 Molybdenum Mo ',42),(' 43 Technetium Tc ',43),(' 44 Ruthenium Ru ',44),
            (' 45 Rhodium Rh ',45),(' 46 Palladium Pd ',46),(' 47 Silver Ag ',47),(' 48 Cadmium Cd ',48),
            (' 49 Indium In ',49),(' 50 Tin Sn ',50),(' 51 Antimony Sb ',51),(' 52 Tellurium Te ',52),
            (' 53 Iodine I ',53),(' 54 Xenon Xe ',54),(' 55 Caesium Cs ',55),(' 56 Barium Ba ',56),
            (' 57 Lanthanum La ',57),(' 58 Cerium Ce ',58),(' 59 Praseodymium Pr ',59),(' 60 Neodymium Nd ',60),
            (' 61 Promethium Pm ',61),(' 62 Samarium Sm ',62),(' 63 Europium Eu ',63),(' 64 Gadolinium Gd ',64),
            (' 65 Terbium Tb ',65),(' 66 Dysprosium Dy ',66),(' 67 Holmium Ho ',67),(' 68 Erbium Er ',68),
            (' 69 Thulium Tm ',69),(' 70 Ytterbium Yb ',70),(' 71 Lutetium Lu ',71),(' 72 Hafnium Hf ',72),
            (' 73 Tantalum Ta ',73),(' 74 Tungsten W ',74),(' 75 Rhenium Re ',75),(' 76 Osmium Os ',76),
            (' 77 Iridium Ir ',77),(' 78 Platinum Pt ',78),(' 79 Gold Au ',79),(' 80 Mercury Hg ',80),
            (' 81 Thallium Tl ',81),(' 82 Lead Pb ',82),(' 83 Bismuth Bi ',83),(' 84 Polonium Po ',84),
            (' 85 Astatine At ',85),(' 86 Radon Rn ',86),(' 87 Francium Fr ',87),(' 88 Radium Ra ',88),
            (' 89 Actinium Ac ',89),(' 90 Thorium Th ',90));

In [86]:
# The periodic table's 90 buttons.  
# Each button includes: atomic symbol, color, layout width and layout height

# 10 colors/element types:alkali metal, alkaline earth, transition metal,basic  
# metal, metalloid, nonmetal, halogen, noble gas, lanthanide, actinide. Plus an  
# unused 11th color - blueviolet.# Stopping at violet causes an out of limit error(??)
clrs = ['tomato', 'orange','yellow','palegreen','springgreen','skyblue','powderblue',
         'cyan','thistle','violet','blueviolet' ] 
# The colors for each of the atoms, 1-90.
clist = [clrs[5],clrs[7],clrs[0],clrs[1],clrs[4],clrs[5],clrs[5],clrs[5],clrs[6],clrs[7],
         clrs[0],clrs[1],clrs[3],clrs[4],clrs[5],clrs[5],clrs[6],clrs[7],clrs[0],clrs[1],
         clrs[2],clrs[2],clrs[2],clrs[2],clrs[2],clrs[2],clrs[2],clrs[2],clrs[2],clrs[2],
         clrs[3],clrs[4],clrs[4],clrs[5],clrs[6],clrs[7],clrs[0],clrs[1],clrs[2],clrs[2],
         clrs[2],clrs[2],clrs[2],clrs[2],clrs[2],clrs[2],clrs[2],clrs[2],clrs[3],clrs[3],
         clrs[4],clrs[4],clrs[6],clrs[7],clrs[0],clrs[1],clrs[8],clrs[8],clrs[8],clrs[8],
         clrs[8],clrs[8],clrs[8],clrs[8],clrs[8],clrs[8],clrs[8],clrs[8],clrs[8],clrs[8],
         clrs[8],clrs[2],clrs[2],clrs[2],clrs[2],clrs[2],clrs[2],clrs[2],clrs[2],clrs[2],
         clrs[3],clrs[3],clrs[3],clrs[4],clrs[6],clrs[7],clrs[0],clrs[1],clrs[9],clrs[9]]
# The atomic symbols 
dlist = ['H','He','Li','Be','B','C','N','O','F','Ne',
         'Na','Mg','Al','Si','P','S','Cl','Ar','K','Ca',
         'Sc','Ti','V','Cr','Mn','Fe','Co','Ni','Cu','Zn',
         'Ga','Ge','As','Se','Br','Kr','Rb','Sr','Y','Zr',
         'Nb','Mo','Tc','Ru','Rh','Pd','Ag','Cd','In','Sn',
         'Sb','Te','I','Xe','Cs','Ba','La','Ce','Pr','Nd',
         'Pm','Sm','Eu','Gd','Tb','Dy','Ho','Er','Tm','Yb',
         'Lu','Hf','Ta','W','Re','Os','Ir','Pt','Au','Hg',
         'Tl','Pb','Bi','Po','At','Rn','Fr','Ra','Ac','Th']
# Defining the buttons by adding the atomic symbols and colors to the buttons
a1 = Button(description=dlist[0], style=dict(button_color=clist[0]))
a2 = Button(description=dlist[1], style=dict(button_color=clist[1]))
a3 = Button(description=dlist[2], style=dict(button_color=clist[2]))
a4 = Button(description=dlist[3], style=dict(button_color=clist[3]))
a5 = Button(description=dlist[4], style=dict(button_color=clist[4]))
a6 = Button(description=dlist[5], style=dict(button_color=clist[5]))
a7 = Button(description=dlist[6], style=dict(button_color=clist[6]))
a8 = Button(description=dlist[7], style=dict(button_color=clist[7]))
a9 = Button(description=dlist[8], style=dict(button_color=clist[8]))
a10 = Button(description=dlist[9], style=dict(button_color=clist[9]))
a11 = Button(description=dlist[10], style=dict(button_color=clist[10]))
a12 = Button(description=dlist[11], style=dict(button_color=clist[11]))
a13 = Button(description=dlist[12], style=dict(button_color=clist[12]))
a14 = Button(description=dlist[13], style=dict(button_color=clist[13]))
a15 = Button(description=dlist[14], style=dict(button_color=clist[15]))
a16 = Button(description=dlist[15], style=dict(button_color=clist[15]))
a17 = Button(description=dlist[16], style=dict(button_color=clist[16]))
a18 = Button(description=dlist[17], style=dict(button_color=clist[17]))
a19 = Button(description=dlist[18], style=dict(button_color=clist[18]))
a20 = Button(description=dlist[19], style=dict(button_color=clist[19]))
a21 = Button(description=dlist[20], style=dict(button_color=clist[20]))
a22 = Button(description=dlist[21], style=dict(button_color=clist[22]))
a23 = Button(description=dlist[22], style=dict(button_color=clist[22]))
a24 = Button(description=dlist[23], style=dict(button_color=clist[23]))
a25 = Button(description=dlist[24], style=dict(button_color=clist[24]))
a26 = Button(description=dlist[25], style=dict(button_color=clist[25]))
a27 = Button(description=dlist[26], style=dict(button_color=clist[26]))
a28 = Button(description=dlist[27], style=dict(button_color=clist[27]))
a29 = Button(description=dlist[28], style=dict(button_color=clist[28]))
a30 = Button(description=dlist[29], style=dict(button_color=clist[29]))
a31 = Button(description=dlist[30], style=dict(button_color=clist[30]))
a32 = Button(description=dlist[31], style=dict(button_color=clist[31]))
a33 = Button(description=dlist[32], style=dict(button_color=clist[32]))
a34 = Button(description=dlist[33], style=dict(button_color=clist[33]))
a35 = Button(description=dlist[34], style=dict(button_color=clist[34]))
a36 = Button(description=dlist[35], style=dict(button_color=clist[35]))
a37 = Button(description=dlist[36], style=dict(button_color=clist[36]))
a38 = Button(description=dlist[37], style=dict(button_color=clist[37]))
a39 = Button(description=dlist[38], style=dict(button_color=clist[38]))
a40 = Button(description=dlist[39], style=dict(button_color=clist[39]))
a41 = Button(description=dlist[40], style=dict(button_color=clist[40]))
a42 = Button(description=dlist[41], style=dict(button_color=clist[41]))
a43 = Button(description=dlist[42], style=dict(button_color=clist[42]))
a44 = Button(description=dlist[43], style=dict(button_color=clist[43]))
a45 = Button(description=dlist[44], style=dict(button_color=clist[44]))
a46 = Button(description=dlist[45], style=dict(button_color=clist[45]))
a47 = Button(description=dlist[46], style=dict(button_color=clist[46]))
a48 = Button(description=dlist[47], style=dict(button_color=clist[47]))
a49 = Button(description=dlist[48], style=dict(button_color=clist[48]))
a50 = Button(description=dlist[49], style=dict(button_color=clist[49]))
a51 = Button(description=dlist[50], style=dict(button_color=clist[50]))
a52 = Button(description=dlist[51], style=dict(button_color=clist[51]))
a53 = Button(description=dlist[52], style=dict(button_color=clist[52]))
a54 = Button(description=dlist[53], style=dict(button_color=clist[53]))
a55 = Button(description=dlist[54], style=dict(button_color=clist[54]))
a56 = Button(description=dlist[55], style=dict(button_color=clist[55]))
a57 = Button(description=dlist[56], style=dict(button_color=clist[56]))
a58 = Button(description=dlist[57], style=dict(button_color=clist[57]))
a59 = Button(description=dlist[58], style=dict(button_color=clist[58]))
a60 = Button(description=dlist[59], style=dict(button_color=clist[59]))
a61 = Button(description=dlist[60], style=dict(button_color=clist[60]))
a62 = Button(description=dlist[61], style=dict(button_color=clist[61]))
a63 = Button(description=dlist[62], style=dict(button_color=clist[62]))
a64 = Button(description=dlist[63], style=dict(button_color=clist[63]))
a65 = Button(description=dlist[64], style=dict(button_color=clist[64]))
a66 = Button(description=dlist[65], style=dict(button_color=clist[65]))
a67 = Button(description=dlist[66], style=dict(button_color=clist[66]))
a68 = Button(description=dlist[67], style=dict(button_color=clist[67]))
a69 = Button(description=dlist[68], style=dict(button_color=clist[68]))
a70 = Button(description=dlist[69], style=dict(button_color=clist[69]))
a71 = Button(description=dlist[70], style=dict(button_color=clist[70]))
a72 = Button(description=dlist[71], style=dict(button_color=clist[71]))
a73 = Button(description=dlist[72], style=dict(button_color=clist[72]))
a74 = Button(description=dlist[73], style=dict(button_color=clist[73]))
a75 = Button(description=dlist[74], style=dict(button_color=clist[74]))
a76 = Button(description=dlist[75], style=dict(button_color=clist[75]))
a77 = Button(description=dlist[76], style=dict(button_color=clist[76]))
a78 = Button(description=dlist[77], style=dict(button_color=clist[77]))
a79 = Button(description=dlist[78], style=dict(button_color=clist[78]))
a80 = Button(description=dlist[79], style=dict(button_color=clist[79]))
a81 = Button(description=dlist[80], style=dict(button_color=clist[80]))
a82 = Button(description=dlist[81], style=dict(button_color=clist[81]))
a83 = Button(description=dlist[82], style=dict(button_color=clist[82]))
a84 = Button(description=dlist[83], style=dict(button_color=clist[83]))
a85 = Button(description=dlist[84], style=dict(button_color=clist[84]))
a86 = Button(description=dlist[85], style=dict(button_color=clist[85]))
a87 = Button(description=dlist[86], style=dict(button_color=clist[86]))
a88 = Button(description=dlist[87], style=dict(button_color=clist[87]))
a89 = Button(description=dlist[88], style=dict(button_color=clist[88]))
a90 = Button(description=dlist[89], style=dict(button_color=clist[89]))

# Creating the list of buttons
abttns = [a1,a2,a3,a4,a5,a6,a7,a8,a9,a10,a11,a12,a13,a14,a15,a16,a17,a18,a19,
               a20,a21,a22,a23,a24,a25,a26,a27,a28,a29,a30,a31,a32,a33,a34,a35,a36,a37,a38,a39,
               a40,a41,a42,a43,a44,a45,a46,a47,a48,a49,a50,a51,a52,a53,a54,a55,a56,a57,a58,a59,
               a60,a61,a62,a63,a64,a65,a66,a67,a68,a69,a70,a71,a72,a73,a74,a75,a76,a77,a78,a79,
               a80,a81,a82,a83,a84,a85,a86,a87,a88,a89,a90]
# Use the button list in a loop to add each button's border, layout width and height.
i=0
while i < 90 :
    abttns[i].layout = widgets.Layout(border = '1px solid black') 
    abttns[i].layout.width = '100%'
    abttns[i].layout.height = '100%'  #'40px'
    i += 1

#a90

In [87]:
# Layout the 90 buttons in periodic table form
grid = GridspecLayout(10, 18)

grid[0,0] = abttns[0]
grid[0,17] = abttns[1]
grid[1,0] = abttns[2]
grid[1,1] = abttns[3]
p=12
while p < 18:
    grid[1,p] = abttns[p-8]
    p += 1
grid[2,0] = abttns[10]
grid[2,1] = abttns[11]
n=12
while n < 18:
    grid[2,n] = abttns[n]
    n += 1
m=0
while m < 18:
    grid[3,m] = abttns[18+m]
    m += 1
k=0
while k < 18:
    grid[4,k] = abttns[36+k]
    k += 1
grid[5,0] = abttns[54]
grid[5,1] = abttns[55]
# grid[5,2] = the Lanthanide series hole
i=3
while i < 18:
    grid[5,i] = abttns[68+i]
    i += 1
grid[6,0] = abttns[86]
grid[6,1] = abttns[87]
# grid[6,2] = the Actinide series hole
j=2
while j < 17:
    # Lanthanide series
    grid[7,j] = abttns[54+j]
    j += 1
#Actinide series
grid[8,2] = abttns[88]
grid[8,3] = abttns[89]


In [88]:
# The periodic table button functions
output = widgets.Output()
display(output)
@output.capture()

def on_a1_clicked(a1): selectAtom.value = 1 
def on_a2_clicked(a2): selectAtom.value = 2 
def on_a3_clicked(a3): selectAtom.value = 3 
def on_a4_clicked(a4): selectAtom.value = 4 
def on_a5_clicked(a5): selectAtom.value = 5 
def on_a6_clicked(a6): selectAtom.value = 6 
def on_a7_clicked(a7): selectAtom.value = 7 
def on_a8_clicked(a8): selectAtom.value = 8 
def on_a9_clicked(a9): selectAtom.value = 9 
def on_a10_clicked(a10): selectAtom.value = 10 
def on_a11_clicked(a11): selectAtom.value = 11 
def on_a12_clicked(a12): selectAtom.value = 12
def on_a13_clicked(a13): selectAtom.value = 13
def on_a14_clicked(a14): selectAtom.value = 14 
def on_a15_clicked(a15): selectAtom.value = 15 
def on_a16_clicked(a16): selectAtom.value = 16 
def on_a17_clicked(a17): selectAtom.value = 17 
def on_a18_clicked(a18): selectAtom.value = 18 
def on_a19_clicked(a19): selectAtom.value = 19 
def on_a20_clicked(a20): selectAtom.value = 20 
def on_a21_clicked(a21): selectAtom.value = 21 
def on_a22_clicked(a22): selectAtom.value = 22 
def on_a23_clicked(a23): selectAtom.value = 23 
def on_a24_clicked(a24): selectAtom.value = 24
def on_a25_clicked(a25): selectAtom.value = 25 
def on_a26_clicked(a26): selectAtom.value = 26
def on_a27_clicked(a27): selectAtom.value = 27
def on_a28_clicked(a28): selectAtom.value = 28
def on_a29_clicked(a29): selectAtom.value = 29 
def on_a30_clicked(a30): selectAtom.value = 30
def on_a31_clicked(a31): selectAtom.value = 31 
def on_a32_clicked(a32): selectAtom.value = 32
def on_a33_clicked(a33): selectAtom.value = 33
def on_a34_clicked(a34): selectAtom.value = 34
def on_a35_clicked(a35): selectAtom.value = 35
def on_a36_clicked(a36): selectAtom.value = 36
def on_a37_clicked(a37): selectAtom.value = 37
def on_a38_clicked(a38): selectAtom.value = 38
def on_a39_clicked(a39): selectAtom.value = 39
def on_a40_clicked(a40): selectAtom.value = 40 
def on_a41_clicked(a41): selectAtom.value = 41 
def on_a42_clicked(a42): selectAtom.value = 42
def on_a43_clicked(a43): selectAtom.value = 43
def on_a44_clicked(a44): selectAtom.value = 44 
def on_a45_clicked(a45): selectAtom.value = 45 
def on_a46_clicked(a46): selectAtom.value = 46 
def on_a47_clicked(a47): selectAtom.value = 47 
def on_a48_clicked(a48): selectAtom.value = 48 
def on_a49_clicked(a49): selectAtom.value = 49
def on_a50_clicked(a50): selectAtom.value = 50
def on_a51_clicked(a51): selectAtom.value = 51 
def on_a52_clicked(a52): selectAtom.value = 52 
def on_a53_clicked(a53): selectAtom.value = 53 
def on_a54_clicked(a54): selectAtom.value = 54 
def on_a55_clicked(a55): selectAtom.value = 55 
def on_a56_clicked(a56): selectAtom.value = 56 
def on_a57_clicked(a57): selectAtom.value = 57 
def on_a58_clicked(a58): selectAtom.value = 58 
def on_a59_clicked(a59): selectAtom.value = 59 
def on_a60_clicked(a59): selectAtom.value = 60 
def on_a61_clicked(a61): selectAtom.value = 61 
def on_a62_clicked(a62): selectAtom.value = 62 
def on_a63_clicked(a63): selectAtom.value = 63 
def on_a64_clicked(a64): selectAtom.value = 64 
def on_a65_clicked(a65): selectAtom.value = 65 
def on_a66_clicked(a66): selectAtom.value = 66 
def on_a67_clicked(a67): selectAtom.value = 67 
def on_a68_clicked(a68): selectAtom.value = 68 
def on_a69_clicked(a69): selectAtom.value = 69 
def on_a70_clicked(a70): selectAtom.value = 70 
def on_a71_clicked(a71): selectAtom.value = 71 
def on_a72_clicked(a72): selectAtom.value = 72 
def on_a73_clicked(a73): selectAtom.value = 73 
def on_a74_clicked(a74): selectAtom.value = 74 
def on_a75_clicked(a75): selectAtom.value = 75 
def on_a76_clicked(a76): selectAtom.value = 76 
def on_a77_clicked(a77): selectAtom.value = 77 
def on_a78_clicked(a78): selectAtom.value = 78 
def on_a79_clicked(a79): selectAtom.value = 79 
def on_a80_clicked(a80): selectAtom.value = 80 
def on_a81_clicked(a81): selectAtom.value = 81 
def on_a82_clicked(a82): selectAtom.value = 82 
def on_a83_clicked(a83): selectAtom.value = 83 
def on_a84_clicked(a84): selectAtom.value = 84 
def on_a85_clicked(a85): selectAtom.value = 85 
def on_a86_clicked(a86): selectAtom.value = 86 
def on_a87_clicked(a87): selectAtom.value = 87 
def on_a88_clicked(a88): selectAtom.value = 88 
def on_a89_clicked(a89): selectAtom.value = 89 
def on_a90_clicked(a90): selectAtom.value = 90     
    
a1.on_click(on_a1_clicked)
a2.on_click(on_a2_clicked)
a3.on_click(on_a3_clicked)
a4.on_click(on_a4_clicked)
a5.on_click(on_a5_clicked)
a6.on_click(on_a6_clicked)
a7.on_click(on_a7_clicked)
a8.on_click(on_a8_clicked)
a9.on_click(on_a9_clicked)
a10.on_click(on_a10_clicked)
a11.on_click(on_a11_clicked)
a12.on_click(on_a12_clicked)
a13.on_click(on_a13_clicked)
a14.on_click(on_a14_clicked)
a15.on_click(on_a15_clicked)
a16.on_click(on_a16_clicked)
a17.on_click(on_a17_clicked)
a18.on_click(on_a18_clicked)
a19.on_click(on_a19_clicked)
a20.on_click(on_a20_clicked)
a21.on_click(on_a21_clicked)
a22.on_click(on_a22_clicked)
a23.on_click(on_a23_clicked)
a24.on_click(on_a24_clicked)
a25.on_click(on_a25_clicked)
a26.on_click(on_a26_clicked)
a27.on_click(on_a27_clicked)
a28.on_click(on_a28_clicked)
a29.on_click(on_a29_clicked)
a30.on_click(on_a30_clicked)
a31.on_click(on_a31_clicked)
a32.on_click(on_a32_clicked)
a33.on_click(on_a33_clicked)
a34.on_click(on_a34_clicked)
a35.on_click(on_a35_clicked)
a36.on_click(on_a36_clicked)
a37.on_click(on_a37_clicked)
a38.on_click(on_a38_clicked)
a39.on_click(on_a39_clicked)
a40.on_click(on_a40_clicked)
a41.on_click(on_a41_clicked)
a42.on_click(on_a42_clicked)
a43.on_click(on_a43_clicked)
a44.on_click(on_a44_clicked)
a45.on_click(on_a45_clicked)
a46.on_click(on_a46_clicked)
a47.on_click(on_a47_clicked)
a48.on_click(on_a48_clicked)
a49.on_click(on_a49_clicked)
a50.on_click(on_a50_clicked)
a51.on_click(on_a51_clicked)
a52.on_click(on_a52_clicked)
a53.on_click(on_a53_clicked)
a54.on_click(on_a54_clicked)
a55.on_click(on_a55_clicked)
a56.on_click(on_a56_clicked)
a57.on_click(on_a57_clicked)
a58.on_click(on_a58_clicked)
a59.on_click(on_a59_clicked)
a60.on_click(on_a60_clicked)
a61.on_click(on_a61_clicked)
a62.on_click(on_a62_clicked)
a63.on_click(on_a63_clicked)
a64.on_click(on_a64_clicked)
a65.on_click(on_a65_clicked)
a66.on_click(on_a66_clicked)
a67.on_click(on_a67_clicked)
a68.on_click(on_a68_clicked)
a69.on_click(on_a69_clicked)
a70.on_click(on_a70_clicked)
a71.on_click(on_a71_clicked)
a72.on_click(on_a72_clicked)
a73.on_click(on_a73_clicked)
a74.on_click(on_a74_clicked)
a75.on_click(on_a75_clicked)
a76.on_click(on_a76_clicked)
a77.on_click(on_a77_clicked)
a78.on_click(on_a78_clicked)
a79.on_click(on_a79_clicked)
a80.on_click(on_a80_clicked)
a81.on_click(on_a81_clicked)
a82.on_click(on_a82_clicked)
a83.on_click(on_a83_clicked)
a84.on_click(on_a84_clicked)
a85.on_click(on_a85_clicked)
a86.on_click(on_a86_clicked)
a87.on_click(on_a87_clicked)
a88.on_click(on_a88_clicked)
a89.on_click(on_a89_clicked)
a90.on_click(on_a90_clicked)


Output()

In [89]:
# Defining the gui widgets
selectAtom = widgets.Dropdown(
    options= elements,
    value=2, 
    description='Atom:',
)
atomsLabel = widgets.Checkbox(
    value=False,
    description='Atomic label',
    disabled=False,
    indent=False
)
emissionType= widgets.Dropdown(
    options=['plane', 'cones'],
    value='plane', 
    description='Emission:',
)
coloredEmissions = widgets.Checkbox(
    value=False,
    description='Color emissions',
    disabled=False,
    indent=False,
)
# Changes the equatorial(or cone) emission radius
emRSlider = widgets.FloatSlider(
    value = 5.0,
    min = 2.0,
    max = 8.0,
    step= 0.1,
    description='EmRadius',
)
# Changes the distance between adjacent orthogonal protons
ePSlider = widgets.FloatSlider(
    value = 5.0,
    min = 2.0,
    max = 7.0,
    step= 0.1,
    description='Equ1_Pole2',
)
# Changes the distance tween top and bottom alpha parallel protons
pPSlider = widgets.FloatSlider(
    value = 4.0,
    min = 3.0,
    max = 6.0,
    step= 0.1,
    description='A width',
)
# Changes the pole to pole distance between 2 adjacent alphas
p2p3Slider = widgets.FloatSlider(
    value = 3.0,
    min = 2.0,
    max = 5.0,
    step= 0.1,
    description='A1_A2 Sep',
)
# Changes the cone emission angle
coneAngleSlider = widgets.FloatSlider(
    value = 30,  # deg  
    min = 20,   
    max = 40,  
    step= 0.1,
    description='Cone angle',
)

guiwidgets = widgets.VBox([selectAtom,atomsLabel,emissionType, # moved
              coloredEmissions,emRSlider,ePSlider,
              pPSlider,p2p3Slider,coneAngleSlider])

In [90]:
# Observe any changes in widget values, button or gui
number = selectAtom.value
def handle_change(change):
    global number
    number = change.new
selectAtom.observe(handle_change, 'value')

emR = emRSlider.value
def handle_change(change):
    global emR
    emR = change.new
emRSlider.observe(handle_change, 'value');

ortho = ePSlider.value # proton1 equator to proton2 pole
def handle_change(change):
    global ortho
    ortho = change.new
ePSlider.observe(handle_change, 'value');

pPTP = pPSlider.value  # proton pole to proton pole 
def handle_change(change):
    global pPTP
    pPTP = change.new
pPSlider.observe(handle_change, 'value')

p2p3Dist = p2p3Slider.value # between 2 adjacent alphas
def handle_change(change):
    global p2p3Dist
    p2p3Dist = change.new
p2p3Slider.observe(handle_change, 'value')

cAngleDeg = coneAngleSlider.value # cone angle
def handle_change(change):
    global cAngleDeg
    cAngleDeg = change.new
coneAngleSlider.observe(handle_change, 'value')


**The Periodic Table.** 
The periodic table below is provided as a sensible way to select 
the desired atom, just click on it. The many atomic properties 
the table is known to provide may be taken into consideration 
while making a choice. 

Most of the controls will be displayed in the cell following the 
periodic table. Note that within those controls, the atom can also 
be selected by a dropdown widget.

Other control options include: type of proton emissions displayed, 
color coding of the specific number of protons in each proton 
stack, proton separation distances, and whether to include an 
atomic label or not.

In [93]:
grid # Display the Periodic Table

GridspecLayout(children=(Button(description='H', layout=Layout(border='1px solid black', grid_area='widget001'…

In [94]:
guiwidgets # Display the main control widgets. 

In [95]:
# This short vertical extra notebook cell allows a better view of the periodic 
An = elements[number-1] # table and guiwidgets than the very long cell below.
atomicLabel = An[0] # 

In [96]:

atomicNumber = number  # Select the desired atom from the dropdown widget
grpAn = Group()  # The rotation widget spins grpAn, slots 1,4 and 5.
grpBn = Group()  # grpBn are the orthogonal top, bottom and hook slots.

if 1 <= number < 13:
    caroSecs = 2.5
if 14 <= number < 20:
    caroSecs = 4.0
if 21 <= number :
    caroSecs = 6.0

slotlist = [0,0,0, 0,0,0, 0,0,0, 0,0,0, 0,0,0, 0,0,0, 0]

# Dimensions # pRadius is now in In[1]
nRadius = pRadius*1.125 # Neutron radius
eRadius = 2*pRadius/10 # Electron radius

# Widget Controlled variables 1. ortho, 2.pPTP, 3. p2p3Dist, 4. cAngleDeg
orthoP = ortho*pRadius
emRadius = 2*emR*pRadius # 2*orthoP - pRadius/2
coneAngle = cAngleDeg*2*piG/360
coneHeight = emRadius*tan(coneAngle)
pP1pP2 = 2*pPTP*pRadius # Widget control.
p2p3 = 2*p2p3Dist*pRadius # Widget control
n1n2 = (pP1pP2- 2*nRadius)/tan(coneAngle) # neutron-neutron orbital separation (y) inside the alpha
# e1e2 = -n1n2/10
p1p2 = pRadius*2.5  #0.75*pP1pP2 # pRadius*2.5
p1n1 = pP1pP2/2 #pP1pP2/2 #pP1pP2/2 # Alpha, up-down (z) proton-orbital neutron separation
p1n2 = pP1pP2/2 #pRadius + nRadius + 1.5*pRadius  # In line proton and neutron
p1e1 = pRadius + 0.05*pP1pP2 # Alpha, top-bottom (z) proton-electron separation
maxZNeg = 0.0 # maxZNeg and maxZPos are Used to center(z) the atom on the screen
maxZPos = 0.0

# Proton set Geometries and Materials
protonGeometry =  SphereBufferGeometry(pRadius, 24, 16)
neutronGeometry =  SphereBufferGeometry(nRadius, 24, 16)
electronGeometry =  SphereBufferGeometry(eRadius, 8, 16)
emissionGeometryP = CircleGeometry(emRadius, 24)
emissionGeometryX = ConeGeometry( emRadius, coneHeight, 16, openEnded = True )
protonMaterial = MeshStandardMaterial(color='blue')
protonMaterialB = MeshStandardMaterial(color='green')
neutronMaterial = MeshStandardMaterial(color='red')
neutronMaterialB = MeshStandardMaterial(color='green') # Color.js Find a nice color spectrum
electronMaterial = MeshStandardMaterial(color='cyan') #'orangered','salmon','green','cyan','teal','orange'
emissionmaterial7= MeshStandardMaterial(color='gray',side='DoubleSide',transparent = True,opacity = 0.20)
emissionmaterial1 = MeshStandardMaterial(color='magenta',side='DoubleSide',transparent = True,opacity = 0.20)
emissionmaterial2 = MeshStandardMaterial(color='blue',side='DoubleSide',transparent = True,opacity = 0.20)
emissionmaterial3 = MeshStandardMaterial(color='pink',side='DoubleSide',transparent = True,opacity = 0.20)
emissionmaterial4 = MeshStandardMaterial(color='salmon',side='DoubleSide',transparent = True,opacity = 0.20)
emissionmaterial5 = MeshStandardMaterial(color='lime',side='DoubleSide',transparent = True,opacity = 0.20)
emissionmaterial6 = MeshStandardMaterial(color='skyblue',side='DoubleSide',transparent = True,opacity = 0.20)

def buildProtonStack(a,j): #  a is the total # of protons. j indicates the calling register
    """ The odd 'a' returns just the single, first, third, or fifth proton. The even 'a'
     returns the first, second or third alpha set, each 2 protons and 2 neutrons. """
    num = 0
    if coloredEmissions.value == True:
        if a == 1 : emissionmaterial = emissionmaterial1
        if a == 2 : emissionmaterial = emissionmaterial2
        if a == 3 : emissionmaterial = emissionmaterial3
        if a == 4 : emissionmaterial = emissionmaterial4
        if a == 5 : emissionmaterial = emissionmaterial5
        if a == 6 : emissionmaterial = emissionmaterial6
    else:
        emissionmaterial = emissionmaterial7
        # Proton z positions: # The possible formula solution, rather than the list, may be in these 2 lines 
        # protonMi.position = [0,0,a*pP1pP2/2+(a-1)*p2p3/2-(p2p3/2+pP1pP2/2)*num] # multiAlpha solution - Not
        # protonMi.position = [0,0,(a//2)*pP1pP2/2+(a//2)*p2p3/2-(p2p3/2+pP1pP2/2)*num] # pSet
    if a == 1 : pPositions = [0]
    if a == 2 : pPositions = [pP1pP2/2,-pP1pP2/2]
    if a == 3 : pPositions = [pP1pP2/2+p2p3/2,-pP1pP2/2+p2p3/2,-pP1pP2/2-p2p3/2]
    if a == 4 : pPositions = [pP1pP2+p2p3/2,p2p3/2,-p2p3/2,-pP1pP2-p2p3/2]
    if a == 5 : pPositions = [pP1pP2+p2p3,p2p3,0,-pP1pP2,-pP1pP2-p2p3]
    if a == 6 : pPositions = [3*pP1pP2/2+p2p3,pP1pP2/2+p2p3,pP1pP2/2,-pP1pP2/2,-pP1pP2/2-p2p3,-3*pP1pP2/2-p2p3]
    while num < a:
        protonMi = Group()
        protonMi.add(Mesh(protonGeometry, protonMaterial))
        if emissionType.value == 'plane':
            protonMi.add(Mesh(emissionGeometry, emissionmaterial))
        if emissionType.value == 'cones':
            emission1 = Object3D()
            emission1.add(Mesh(emissionGeometry, emissionmaterial))
            emission1.position = [0,0,coneHeight/2]
            emission1.rotateX(-piG/2)
            protonMi.add(emission1)
            emission2 = Object3D()
            emission2.add(Mesh(emissionGeometry, emissionmaterial))
            emission2.position = [0,0,-coneHeight/2]
            emission2.rotateX(piG/2)
            protonMi.add(emission2)
            protonMi.add(Mesh(emissionGeometryP, emissionmaterial))
        electronMi = Object3D()
        electronMi.add(Mesh(electronGeometry, electronMaterial))
        electronMi.position = [0,-n1n2/10,-p1e1]
        protonMi.add(electronMi)
        protonMi.rotateZ(random()*2*piG)
        if num%2 != 0:
            neutronPair = Group()
            neutronMi = Object3D()
            neutronMi.add(Mesh(neutronGeometry, neutronMaterial))
            neutronMi.position = [0,-n1n2/2,-p1n1]
            neutronNi = Object3D()
            neutronNi.add(Mesh(neutronGeometry, neutronMaterial))
            neutronNi.position = [0,n1n2/2,-p1n1]
            neutronPair.add(neutronMi)
            neutronPair.add(neutronNi)
            neutronPair.rotateZ(random()*2*piG)
            protonMi.add(neutronPair)
            protonMi.rotateX(piG)
            protonMi.rotateZ(random()*2*piG)
        protonMi.position = [0,0,pPositions[num]]
        if j == 0 : proStack1.add(protonMi)
        if j == 1 : proStack2.add(protonMi)
        if j == 2 : proStack3.add(protonMi)
        if j == 3 : proStack4.add(protonMi)
        if j == 4 : proStack5.add(protonMi)
        if j == 5 : proStack6.add(protonMi)
        if j == 6 : proStack7.add(protonMi)
        if j == 7 : proStack8.add(protonMi)
        if j == 8 : proStack9.add(protonMi)
        if j == 9 : proStack10.add(protonMi)
        if j == 10 : proStack11.add(protonMi)
        if j == 11 : proStack12.add(protonMi)
        if j == 12 : proStack13.add(protonMi)
        if j == 13 : proStack14.add(protonMi)
        if j == 14 : proStack15.add(protonMi)
        if j == 15 : proStack16.add(protonMi)
        if j == 16 : proStack17.add(protonMi)
        if j == 17 : proStack18.add(protonMi)
        if j == 18 : proStack19.add(protonMi)
        num += 1
    return

# Assemble the atomic models.
for i in range( atomicNumber ):
    if emissionType.value == 'plane':
        emissionGeometry = emissionGeometryP
    if emissionType.value == 'cones':
        emissionGeometry = emissionGeometryX
    if coloredEmissions.value == True:
        emissionmaterial = emissionmaterial1
    else:
        emissionmaterial = emissionmaterial7
    neutronGi = Object3D()
    neutronGi.add(Mesh(neutronGeometry, neutronMaterial))
    neutronKi = Object3D()
    neutronKi.add(Mesh(neutronGeometry, neutronMaterial))
    neutronLi = Object3D()
    neutronLi.add(Mesh(neutronGeometry, neutronMaterial))

    if i == 0: # Slot 1
        proStack1 = Group()
        if 1 == atomicNumber: # Hydrogen. Slot 1
            buildProtonStack(1,i)
            neutronGi.position = [0,0,-p1n2]
            proStack1.position = [0, 0, 0]
            proStack1.add(neutronGi)
            grpAn.add(proStack1)
            slotlist[i] = 1
            #print('proStack1 1[',i+1,']', proStack1.position)
        if 2 <= atomicNumber < 6: # Helium
            buildProtonStack(2,i)
            grpAn.add(proStack1)
            slotlist[i] = 2
            #print('proStack1 2[',i+1,']', proStack1.position)
        if 6 == atomicNumber :
            buildProtonStack(4,i)
            grpAn.add(proStack1)
            slotlist[i] = 4
            maxZNeg = -3*pP1pP2/2-2*orthoP
            maxZPos = 3*pP1pP2/2+2*orthoP
            #print('proStack1 4[',i+1,']', proStack1.position)
        if 7 <= atomicNumber < 10 :
            buildProtonStack(6,i)
            grpAn.add(proStack1)
            slotlist[i] = 6
            maxZNeg = -3*pP1pP2/2-p2p3
            maxZPos = p2p3+3*pP1pP2/2+2*orthoP
            #print('proStack1 6[',i+1,']', proStack1.position)
        if 10 <= atomicNumber < 36 :
            buildProtonStack(2,i)
            grpAn.add(proStack1)
            slotlist[i] = 2
            #print('proStack1 2[',i+1,']', proStack1.position)
        if 36 <= atomicNumber < 54 :
            buildProtonStack(4,i)
            grpAn.add(proStack1)
            slotlist[i] = 4
            #print('proStack1 4[',i+1,']', proStack1.position)
        if 54 <= atomicNumber < 57:
            buildProtonStack(6,i)
            grpAn.add(proStack1)
            slotlist[i] = 6
            #print('proStack1 6[',i+1,']', proStack1.position)
        if 57 <= atomicNumber < 71:
            buildProtonStack(5,i)            
            grpAn.add(proStack1)
            slotlist[i] = 5
            #print('proStack1 5[',i+1,']', proStack1.position)
        if 71 <= atomicNumber :
            buildProtonStack(6,i)
            grpAn.add(proStack1)
            slotlist[i] = 6
            #print('proStack1 6[',i+1,']', proStack1.position)

    if i == 1: # Slot 2
        proStack2 = Group()
        if 2 < atomicNumber < 6:
            buildProtonStack(1,i)
            proStack2.rotateX(-piG/2)
            proStack2.position = [0, 0, pP1pP2/2 + 2*orthoP]
            grpBn.add(proStack2)
            slotlist[i] = 1
            #print('proStack2 1[',i+1,']', proStack2.position)
        if 6 == atomicNumber :
            buildProtonStack(1,i)
            proStack2.rotateX(-piG/2)
            proStack2.position = [0, 0,p2p3/2+pP1pP2+2*orthoP]
            grpBn.add(proStack2)
            slotlist[i] = 1
            #print('proStack2 1[',i+1,']', proStack2.position)
        if 7 <= atomicNumber < 10:
            buildProtonStack(1,i)
            proStack2.rotateX(-piG/2)
            proStack2.position = [0, 0,p2p3+3*pP1pP2/2+2*orthoP]
            grpBn.add(proStack2)
            slotlist[i] = 1
            #print('proStack2 1[',i+1,']', proStack2.position)
        if 10 <= atomicNumber < 36 :
            buildProtonStack(2,i)
            proStack2.rotateX(-piG/2)
            proStack2.position = [0, 0, pP1pP2/2 + 2*orthoP] 
            grpBn.add(proStack2)
            slotlist[i] = 2
            #print('proStack2 2[',i+1,']', proStack2.position)
        if 36 <= atomicNumber < 54:
            buildProtonStack(4,i)
            proStack2.rotateX(piG/2)
            proStack2.position = [0,0,p1p2/2+pP1pP2+2*orthoP]
            grpBn.add(proStack2)
            slotlist[i] = 4
            #print('proStack2 4[',i+1,']', proStack2.position)
        if 54 <= atomicNumber < 57 :
            buildProtonStack(6,i)
            proStack2.rotateX(piG/2)
            proStack2.position = [0,0,p2p3+3*pP1pP2/2+2*orthoP]
            grpBn.add(proStack2)
            slotlist[i] = 6
            #print('proStack2 6[',i+1,']', proStack2.position)
        if 57 <= atomicNumber < 71 :
            buildProtonStack(5,i)
            proStack2.rotateX(piG/2)
            proStack2.position = [0,0,p2p3+pP1pP2+2*orthoP]
            grpBn.add(proStack2)
            slotlist[i] = 5
            #print('proStack2 5[',i+1,']', proStack2.position)
        if 71 <= atomicNumber :
            buildProtonStack(6,i)
            proStack2.rotateX(piG/2)
            proStack2.position = [0,0,p2p3+3*pP1pP2/2+2*orthoP]
            grpBn.add(proStack2)
            slotlist[i] = 6
            #print('proStack2 6[',i+1,']', proStack2.position)

    if i == 2: # Slot 3
        proStack3 = Group()
        if 3 < atomicNumber < 5:
            buildProtonStack(1,i)
            proStack3.rotateX(-piG/2)
            proStack3.position = [0, 0, -pP1pP2/2 - 2*orthoP]
            grpBn.add(proStack3)
            slotlist[i] = 1
            #print('proStack3 1[',i+1,']', proStack3.position)
        if 5 == atomicNumber :
            buildProtonStack(2,i)
            proStack3.rotateX(-piG/2)
            proStack3.position = [0, 0, -pP1pP2/2 - 2*orthoP]
            grpBn.add(proStack3)
            slotlist[i] = 2
            #print('proStack3 2[',i+1,']', proStack3.position)
        if 6 == atomicNumber :
            buildProtonStack(1,i)
            proStack3.rotateX(-piG/2)
            proStack3.position = [0, 0, -p2p3/2-pP1pP2-2*orthoP]
            grpBn.add(proStack3)
            slotlist[i] = 1
            #print('proStack3 1[',i+1,']', proStack3.position)
        if 8 == atomicNumber :
            buildProtonStack(1,i)
            proStack3.rotateX(-piG/2)
            proStack3.position = [0, 0,-3*pP1pP2/2-p2p3-2*orthoP]
            grpBn.add(proStack3)
            slotlist[i] = 1
            maxZNeg = -3*pP1pP2/2-p2p3-2*orthoP
            maxZPos = 3*pP1pP2/2+p2p3+2*orthoP
            #print('proStack3 1[',i+1,']', proStack3.position)
        if 9 == atomicNumber :
            buildProtonStack(2,i)
            proStack3.rotateX(-piG/2)
            proStack3.position = [0, 0,-3*pP1pP2/2-p2p3-2*orthoP]
            grpBn.add(proStack3)
            slotlist[i] = 2
            maxZNeg = -3*pP1pP2/2-p2p3-2*orthoP
            maxZPos = 3*pP1pP2/2+p2p3+2*orthoP
            #print('proStack3 2[',i+1,']', proStack3.position)
        if 10 <= atomicNumber < 36 :
            buildProtonStack(2,i)
            proStack3.rotateX(-piG/2)
            proStack3.position = [0, 0, -pP1pP2/2 - 2*orthoP]
            grpBn.add(proStack3)
            slotlist[i] = 2
            #print('proStack3 2[',i+1,']', proStack3.position)
        if 36 <= atomicNumber < 54 :
            buildProtonStack(4,i)
            proStack3.rotateX(piG/2)
            proStack3.position = [0,0,-p1p2/2-pP1pP2-2*orthoP]
            grpBn.add(proStack3)
            slotlist[i] = 4
            #print('proStack3 4[',i+1,']', proStack3.position)
        if 54 <= atomicNumber < 57 :
            buildProtonStack(6,i)
            proStack3.rotateX(piG/2)
            proStack3.position = [0,0,-p2p3-3*pP1pP2/2-2*orthoP]
            grpBn.add(proStack3)
            slotlist[i] = 6
            #print('proStack3 6[',i+1,']', proStack3.position)
        if 57 <= atomicNumber < 71 :
            buildProtonStack(5,i)
            proStack3.rotateX(piG/2)
            proStack3.position = [0,0,-p2p3-pP1pP2-2*orthoP]
            grpBn.add(proStack3)
            slotlist[i] = 5
            #print('proStack3 5[',i+1,']', proStack3.position)
        if 71 <= atomicNumber :
            buildProtonStack(6,i)
            proStack3.rotateX(piG/2)
            proStack3.position = [0,0,-p2p3-3*pP1pP2/2-2*orthoP]
            grpBn.add(proStack3)
            slotlist[i] = 6
            #print('proStack3 6[',i+1,']', proStack3.position)

    if i == 3: # Slot 4
        proStack4 = Group()
        if 10 <= atomicNumber < 14: # Neon
            buildProtonStack(2,i)
            proStack4.position = [0, 0, pP1pP2 + 4*orthoP]
            grpBn.add(proStack4)
            slotlist[i] = 2
            #print('proStack4 2[',i+1,']', proStack4.position)
        if 15 == atomicNumber :
            buildProtonStack(2,i)
            proStack4.position = [0, 0, pP1pP2 + 4*orthoP]
            grpBn.add(proStack4)
            #print('proStack4 2[',i+1,']', proStack4.position)
            slotlist[i] = 2
            maxZNeg = -3*pP1pP2/2-6*orthoP
            maxZPos = 3*pP1pP2/2+4*orthoP
        if 16 == atomicNumber :
            buildProtonStack(1,i)
            proStack4.position = [0, 0, pP1pP2/2 + 4*orthoP]
            grpBn.add(proStack4)
            slotlist[i] = 1
            #print('proStack4 1[',i+1,']', proStack4.position)
            maxZNeg = -pP1pP2/2-4*orthoP
            maxZPos = 3*pP1pP2/2+6*orthoP
        if 17 <= atomicNumber < 36 :
            buildProtonStack(2,i)
            proStack4.position = [0, 0, pP1pP2 + 4*orthoP]
            grpBn.add(proStack4)
            slotlist[i] = 2
            #print('proStack4 2[',i+1,']', proStack4.position)
        if 36 <= atomicNumber < 54:
            buildProtonStack(4,i)
            proStack4.position = [0,0,p1p2/2+pP1pP2+p1p2/2+pP1pP2+4*orthoP]
            grpBn.add(proStack4)
            slotlist[i] = 4
            #print('proStack4 4[',i+1,']', proStack4.position)
        if 54 <= atomicNumber < 57:
            buildProtonStack(6,i)
            proStack4.position = [0,0,2*p2p3+3*pP1pP2+4*orthoP]
            grpBn.add(proStack4)
            slotlist[i] = 6
            #print('proStack4 6[',i+1,']', proStack4.position)
        if 57 <= atomicNumber < 71:
            buildProtonStack(5,i)
            proStack4.position = [0,0,2*p2p3+2*pP1pP2+4*orthoP]
            grpBn.add(proStack4)
            slotlist[i] = 5
            #print('proStack4 5[',i+1,']', proStack4.position)
        if 71 <= atomicNumber :
            buildProtonStack(6,i)
            proStack4.position = [0,0,2*p2p3+3*pP1pP2+4*orthoP]
            grpBn.add(proStack4)
            slotlist[i] = 6
            #print('proStack4 6[',i+1,']', proStack4.position)

    if i == 4: # Slot 5
        proStack5 = Group()
        if 10 <= atomicNumber < 14 :
            buildProtonStack(2,i)
            proStack5.position = [0, 0, -pP1pP2 - 4*orthoP]
            grpBn.add(proStack5)
            slotlist[i] = 2
            #print('proStack5 2[',i+1,']', proStack5.position)
        if 15 == atomicNumber :
            buildProtonStack(2,i)
            proStack5.position = [0, 0, -pP1pP2 - 4*orthoP]
            grpBn.add(proStack5)
            slotlist[i] = 2
            #print('proStack5 2[',i+1,']', proStack5.position)
        if 16 == atomicNumber :
            buildProtonStack(1,i)
            proStack5.position = [0, 0, -pP1pP2/2 - 4*orthoP]
            grpBn.add(proStack5)
            slotlist[i] = 1
            #print('proStack5 1[',i+1,']', proStack5.position)
            maxZNeg = -pP1pP2/2-4*orthoP
            maxZPos = 3*pP1pP2/2+6*orthoP
        if 17 <= atomicNumber < 21 :
            buildProtonStack(2,i)
            proStack5.position = [0, 0, -pP1pP2 - 4*orthoP]
            grpBn.add(proStack5)
            slotlist[i] = 2
            #print('proStack5 2[',i+1,']', proStack5.position)
        if 21 == atomicNumber :
            buildProtonStack(2,i)
            neutronGi.position = [0, 0, -pP1pP2/2 - p1n2]
            proStack5.add(neutronGi)
            proStack5.position = [0, 0, -pP1pP2 - 4*orthoP]
            grpBn.add(proStack5)
            slotlist[i] = 2
            #print('proStack5 2 w neutron[',i+1,']', proStack5.position)
        if 22 <= atomicNumber < 36 :
            buildProtonStack(2,i)
            proStack5.position = [0, 0, -pP1pP2 - 4*orthoP]
            grpBn.add(proStack5)
            slotlist[i] = 2
            #print('proStack5 2[',i+1,']', proStack5.position)
        if 36 <= atomicNumber < 54:
            proStack5 = Group()
            buildProtonStack(4,i)
            proStack5.position = [0,0,-p1p2/2-pP1pP2-p1p2/2-pP1pP2-4*orthoP]
            grpBn.add(proStack5)
            slotlist[i] = 4
            #print('proStack5 4[',i+1,']', proStack5.position)
        if 54 <= atomicNumber < 57:
            buildProtonStack(6,i)
            proStack5.position = [0,0,-2*p2p3-3*pP1pP2-4*orthoP]
            grpBn.add(proStack5)
            slotlist[i] = 6
            #print('proStack5 6[',i+1,']', proStack5.position)
        if 57 <= atomicNumber < 71:
            buildProtonStack(5,i)
            proStack5.position = [0,0,-2*p2p3-2*pP1pP2-4*orthoP]
            grpBn.add(proStack5)
            slotlist[i] = 5
            #print('proStack5 5[',i+1,']', proStack5.position)
        if 71 <= atomicNumber :
            buildProtonStack(6,i)
            proStack5.position = [0,0,-2*p2p3-3*pP1pP2-4*orthoP]
            grpBn.add(proStack5)
            slotlist[i] = 6
            #print('proStack5 6[',i+1,']', proStack5.position)

    if i == 5: # Slot 6
        proStack6 = Group()
        if 11 <= atomicNumber  < 14 :
            buildProtonStack(1,i)
            proStack6.rotateX(-piG/2)
            proStack6.position = [0,0, 3*pP1pP2/2+6*orthoP]
            grpBn.add(proStack6)
            slotlist[i] = 1
            #print('proStack6 1[',i+1,']', proStack6.position)
        if 17 <= atomicNumber < 18 :
            buildProtonStack(2,i)
            proStack6.rotateX(-piG/2)
            proStack6.position = [0, 0, 3*pP1pP2/2+6*orthoP]
            grpBn.add(proStack6)
            slotlist[i] = 2
            #print('proStack6 2[',i+1,']', proStack6.position)
            maxZNeg = -3*pP1pP2/2-6*orthoP
            maxZPos = 3*pP1pP2/2+6*orthoP
        if 19 <= atomicNumber < 22 :
            buildProtonStack(1,i)
            proStack6.rotateX(-piG/2)
            proStack6.position = [0, 0, 3*pP1pP2/2+6*orthoP]
            grpBn.add(proStack6)
            slotlist[i] = 1
            #print('proStack6 1[',i+1,']', proStack6.position)
            maxZNeg = -3*pP1pP2-4*orthoP
            maxZPos = 3*pP1pP2/2+6*orthoP
        if 23 <= atomicNumber < 26 :
            buildProtonStack(1,i)
            proStack6.rotateX(-piG/2)
            proStack6.position = [0, 0, 3*pP1pP2/2+6*orthoP]
            grpBn.add(proStack6)
            slotlist[i] = 1
            #print('proStack6 1[',i+1,']', proStack6.position)
            maxZNeg = -3*pP1pP2-4*orthoP
            maxZPos = 3*pP1pP2/2+6*orthoP
        if 26 <= atomicNumber < 29:
            buildProtonStack(2,i)
            proStack6.rotateX(-piG/2)
            proStack6.position = [0, 0, 3*pP1pP2/2+6*orthoP]
            grpBn.add(proStack6)
            slotlist[i] = 2
            #print('proStack6 2[',i+1,']', proStack6.position)
        if 29 <= atomicNumber < 31:
            buildProtonStack(1,i)
            proStack6.rotateX(-piG/2)
            proStack6.position = [0, 0, 3*pP1pP2/2+6*orthoP]
            grpBn.add(proStack6)
            slotlist[i] = 1
            #print('proStack6 2[',i+1,']', proStack6.position)
            maxZNeg = -3*pP1pP2-4*orthoP
            maxZPos = 3*pP1pP2/2+6*orthoP
        if 31 <= atomicNumber < 36 :
            buildProtonStack(2,i)
            proStack6.rotateX(-piG/2)
            proStack6.position = [0, 0, 3*pP1pP2/2+6*orthoP]
            grpBn.add(proStack6)
            slotlist[i] = 2
            #Zprint('proStack6 2[',i+1,']', proStack6.position)
        if 37 <= atomicNumber < 48:
            buildProtonStack(1,i)
            proStack6.rotateX(-piG/2)
            proStack6.position = [0,0,3*p1p2/2+6*pP1pP2/2+6*orthoP]
            grpBn.add(proStack6)
            slotlist[i] = 1
            #print('proStack6 1[',i+1,']', proStack6.position)
            maxZNeg = -3*p1p2/2-5*pP1pP2/2-4*orthoP
            maxZPos = 3*p1p2/2+6*pP1pP2/2+6*orthoP
        if 48 <= atomicNumber < 50:
            buildProtonStack(2,i)
            proStack6.rotateX(-piG/2)
            proStack6.position = [0, 0, 3*p1p2/2+6*pP1pP2/2+6*orthoP]
            grpBn.add(proStack6)
            slotlist[i] = 2
            #print('proStack6 2[',i+1,']', proStack6.position)
        if 50 == atomicNumber:
            buildProtonStack(1,i)
            proStack6.rotateX(-piG/2)
            proStack6.position = [0,0,3*p1p2/2+6*pP1pP2/2+6*orthoP]
            grpBn.add(proStack6)
            slotlist[i] = 1
            #print('proStack6 1[',i+1,']', proStack6.position)
        if 51 <= atomicNumber < 54:
            buildProtonStack(2,i)
            proStack6.rotateX(-piG/2)
            proStack6.position = [0, 0, 3*p1p2/2+6*pP1pP2/2+6*orthoP]
            grpBn.add(proStack6)
            slotlist[i] = 2
            #print('proStack6 2[',i+1,']', proStack6.position)
        if 55 <= atomicNumber < 57 :
            buildProtonStack(1,i)
            proStack6.rotateX(-piG/2)
            proStack6.position = [0, 0,3*p2p3+4.5*pP1pP2+6*orthoP]
            grpBn.add(proStack6)
            slotlist[i] = 1
            #print('proStack6 1[',i+1,']', proStack6.position)
            maxZNeg = -3*p1p2/2-5*pP1pP2/2-4*orthoP
            maxZPos = 3*p1p2/2+6*pP1pP2/2+6*orthoP
        if 57 <= atomicNumber < 62:
            buildProtonStack(1,i)
            proStack6.rotateX(-piG/2)
            proStack6.position = [0, 0,3*p2p3+3*pP1pP2+6*orthoP]
            grpBn.add(proStack6)
            slotlist[i] = 1
            #print('proStack6 1[',i+1,']', proStack6.position)
        if 62 <= atomicNumber < 66 :
            buildProtonStack(2,i)
            proStack6.rotateX(-piG/2)
            proStack6.position = [0, 0,3*p2p3+3*pP1pP2+6*orthoP]
            grpBn.add(proStack6)
            slotlist[i] = 2
            #print('proStack6 2[',i+1,']', proStack6.position)
        if 66 <= atomicNumber < 71:
            buildProtonStack(3,i)
            proStack6.rotateX(-piG/2)
            proStack6.position = [0, 0,3*p2p3+3*pP1pP2+6*orthoP]
            grpBn.add(proStack6)
            slotlist[i] = 3
            #print('proStack6 3[',i+1,']', proStack6.position)
        if 71 == atomicNumber :
            buildProtonStack(2,i)
            proStack6.rotateX(-piG/2)
            proStack6.position = [0, 0,3*p2p3+4.5*pP1pP2+6*orthoP]
            grpBn.add(proStack6)
            slotlist[i] = 2
            #print('proStack6 2[',i+1,']', proStack6.position)
        if 72 <= atomicNumber < 78 :
            buildProtonStack(3,i)
            proStack6.rotateX(-piG/2)
            proStack6.position = [0, 0,3*p2p3+4.5*pP1pP2+6*orthoP]
            grpBn.add(proStack6)
            slotlist[i] = 3
            print('proStack6 3[',i+1,']', proStack6.position)
        if 78 == atomicNumber :
            buildProtonStack(4,i)
            proStack6.rotateX(-piG/2)
            proStack6.position = [0, 0,3*p2p3+4.5*pP1pP2+6*orthoP]
            grpBn.add(proStack6)
            slotlist[i] = 4
            #print('proStack6 4[',i+1,']', proStack6.position)
        if 79 == atomicNumber :
            buildProtonStack(3,i)
            proStack6.rotateX(-piG/2)
            proStack6.position = [0, 0,3*p2p3+4.5*pP1pP2+6*orthoP]
            grpBn.add(proStack6)
            slotlist[i] = 3
            #print('proStack6 3[',i+1,']', proStack6.position)
        if 80 <= atomicNumber < 82 :
            buildProtonStack(4,i)
            proStack6.rotateX(-piG/2)
            proStack6.position = [0, 0,3*p2p3+4.5*pP1pP2+6*orthoP]
            grpBn.add(proStack6)
            slotlist[i] = 4
            #print('proStack6 4[',i+1,']', proStack6.position)
        if 82 <= atomicNumber :
            buildProtonStack(5,i)
            proStack6.rotateX(-piG/2)
            proStack6.position = [0, 0,3*p2p3+4.5*pP1pP2+6*orthoP]
            grpBn.add(proStack6)
            slotlist[i] = 5
            #print('proStack6 5[',i+1,']', proStack6.position)

    if i == 6: # Slot 7
        proStack7 = Group()
        if 12 == atomicNumber :
            buildProtonStack(1,i)
            proStack7.rotateX(-piG/2)
            proStack7.position = [0,0, -3*pP1pP2/2-6*orthoP]
            grpBn.add(proStack7)
            slotlist[i] = 1
            #print('proStack7 1[',i+1,']', proStack7.position)
        if 15 == atomicNumber:
            buildProtonStack(1,i)
            proStack7.rotateX(-piG/2)
            proStack7.position = [0, 0, -3*pP1pP2/2-6*orthoP]
            grpBn.add(proStack7)
            slotlist[i] = 1
            #print('proStack7 1[',i+1,']', proStack7.position)
        if 17 == atomicNumber:
            buildProtonStack(1,i)
            proStack7.rotateX(-piG/2)
            proStack7.position = [0, 0, -3*pP1pP2/2-6*orthoP]
            grpBn.add(proStack7)
            slotlist[i] = 1
            #print('proStack7 1[',i+1,']', proStack7.position)
        if 20 == atomicNumber:
            buildProtonStack(1,i)
            proStack7.rotateX(-piG/2)
            proStack7.position = [0, 0, -3*pP1pP2/2-6*orthoP]
            grpBn.add(proStack7)
            slotlist[i] = 1
            maxZNeg = -3*pP1pP2/2-6*orthoP
            maxZPos = 3*pP1pP2/2+6*orthoP
            #print('proStack7 1[',i+1,']', proStack7.position)
        if 24 <= atomicNumber < 26:
            buildProtonStack(1,i)
            proStack7.rotateX(-piG/2)
            proStack7.position = [0, 0, -3*pP1pP2/2-6*orthoP]
            grpBn.add(proStack7)
            slotlist[i] = 1
            maxZNeg = -3*pP1pP2/2-6*orthoP
            maxZPos = 3*pP1pP2/2+6*orthoP
            #print('proStack7 1[',i+1,']', proStack7.position)
        if 26 <= atomicNumber < 30 :
            buildProtonStack(2,i)
            proStack7.rotateX(piG/2)
            proStack7.position = [0, 0, -3*pP1pP2/2-6*orthoP]
            grpBn.add(proStack7)
            slotlist[i] = 2
            #print('proStack7 2[',i+1,']', proStack7.position)
        if 30 == atomicNumber :
            buildProtonStack(1,i)
            proStack7.rotateX(piG/2)
            proStack7.position = [0, 0, -3*pP1pP2/2-6*orthoP]
            grpBn.add(proStack7)
            slotlist[i] = 1
            maxZNeg = -3*pP1pP2/2-6*orthoP
            maxZPos = 3*pP1pP2/2+6*orthoP
            #print('proStack7 1[',i+1,']', proStack7.position)
        if 31 <= atomicNumber < 36 :
            buildProtonStack(2,i)
            proStack7.rotateX(piG/2)
            proStack7.position = [0, 0, -3*pP1pP2/2-6*orthoP]
            grpBn.add(proStack7)
            slotlist[i] = 2
            #print('proStack7 2[',i+1,']', proStack7.position)
        if 38 == atomicNumber:
            buildProtonStack(1,i)
            proStack7.rotateX(piG/2)
            proStack7.position = [0,0,-3*p1p2/2-6*pP1pP2/2-6*orthoP]
            grpBn.add(proStack7)
            slotlist[i] = 1
            maxZNeg = -3*p1p2/2-6*pP1pP2/2-6*orthoP
            maxZPos = 3*p1p2/2+6*pP1pP2/2+6*orthoP
            #print('proStack7 1[',i+1,']', proStack7.position)
        if 40 <= atomicNumber < 47:
            buildProtonStack(1,i)
            proStack7.rotateX(piG/2)
            proStack7.position = [0,0,-3*p1p2/2-6*pP1pP2/2-6*orthoP]
            grpBn.add(proStack7)
            slotlist[i] = 1
            maxZNeg = -3*p1p2/2-6*pP1pP2/2-6*orthoP
            maxZPos = 3*p1p2/2+6*pP1pP2/2+6*orthoP
            #print('proStack7 1[',i+1,']', proStack7.position)
        if 47 <= atomicNumber < 50:
            buildProtonStack(2,i)
            proStack7.rotateX(-piG/2)
            proStack7.position = [0,0,-3*p1p2/2-6*pP1pP2/2-6*orthoP]
            grpBn.add(proStack7)
            slotlist[i] = 2
            #print('proStack7 2[',i+1,']', proStack7.position)
            maxZNeg = -3*p1p2/2-6*pP1pP2/2-6*orthoP
            maxZPos = 3*p1p2/2+6*pP1pP2/2+6*orthoP
        if 50 == atomicNumber:
            buildProtonStack(1,i)
            proStack7.rotateX(piG/2)
            proStack7.position = [0,0,-3*p1p2/2-6*pP1pP2/2-6*orthoP]
            grpBn.add(proStack7)
            slotlist[i] = 1
            print('proStack7 1[',i+1,']', proStack7.position)
        if 51 <= atomicNumber < 54:
            buildProtonStack(2,i)
            proStack7.rotateX(-piG/2)
            proStack7.position = [0,0,-3*p1p2/2-6*pP1pP2/2-6*orthoP]
            grpBn.add(proStack7)
            slotlist[i] = 2
            #print('proStack7 2[',i+1,']', proStack7.position)
        if 56 == atomicNumber < 57:
            buildProtonStack(1,i)
            proStack7.rotateX(piG/2)
            proStack7.position = [0, 0,-3*p2p3-4.5*pP1pP2-6*orthoP]
            grpBn.add(proStack7)
            slotlist[i] = 1
            #print('proStack7 1[',i+1,']', proStack7.position)
            maxZNeg = -3*p1p2/2-6*pP1pP2/2-6*orthoP # needed
            maxZPos = 3*p1p2/2+6*pP1pP2/2+6*orthoP
        if 57 <= atomicNumber < 63:
            buildProtonStack(2,i)
            proStack7.rotateX(-piG/2)
            proStack7.position = [0, 0,-3*p2p3-3*pP1pP2-6*orthoP]
            grpBn.add(proStack7)
            slotlist[i] = 2
            #print('proStack7 2[',i+1,']', proStack7.position)
        if 63 <= atomicNumber < 66 :
            buildProtonStack(3,i)
            proStack7.rotateX(-piG/2)
            proStack7.position = [0, 0,-3*p2p3-3*pP1pP2-6*orthoP]
            grpBn.add(proStack7)
            slotlist[i] = 3
            #print('proStack7 3[',i+1,']', proStack7.position)
        if 66 == atomicNumber :
            buildProtonStack(4,i)
            proStack7.rotateX(-piG/2)
            proStack7.position = [0, 0,-3*p2p3-3*pP1pP2-6*orthoP]
            grpBn.add(proStack7)
            slotlist[i] = 4
            #print('proStack7 4[',i+1,']', proStack7.position)
        if 67 == atomicNumber :
            buildProtonStack(3,i)
            proStack7.rotateX(-piG/2)
            proStack7.position = [0, 0,-3*p2p3-3*pP1pP2-6*orthoP]
            grpBn.add(proStack7)
            slotlist[i] = 3
            #print('proStack7 3[',i+1,']', proStack7.position)
        if 68 <= atomicNumber < 71 :
            buildProtonStack(4,i)
            proStack7.rotateX(-piG/2)
            proStack7.position = [0, 0,-3*p2p3-3*pP1pP2-6*orthoP]
            grpBn.add(proStack7)
            slotlist[i] = 4
            #print('proStack7 4[',i+1,']', proStack7.position)
        if 71 <= atomicNumber < 78 :
            buildProtonStack(3,i)
            proStack7.rotateX(-piG/2)
            proStack7.position = [0, 0,-3*p2p3-4.5*pP1pP2-6*orthoP]
            grpBn.add(proStack7)
            slotlist[i] = 3
            #print('proStack7 3[',i+1,']', proStack7.position)
        if 78 <= atomicNumber < 82 :
            buildProtonStack(4,i)
            proStack7.rotateX(-piG/2)
            proStack7.position = [0, 0,-3*p2p3-4.5*pP1pP2-6*orthoP]
            grpBn.add(proStack7)
            slotlist[i] = 4
            #print('proStack7 4[',i+1,']', proStack7.position)
        if 82 <= atomicNumber :
            buildProtonStack(5,i)
            proStack7.rotateX(-piG/2)
            proStack7.position = [0, 0,-3*p2p3-4.5*pP1pP2-6*orthoP]
            grpBn.add(proStack7)
            slotlist[i] = 5
            #print('proStack7 5[',i+1,']', proStack7.position)

    if i == 7: # Slot 8
        proStack8 = Group()
        if 13 == atomicNumber :
            buildProtonStack(1,i)
            proStack8.rotateX(piG/2)
            proStack8.position = [0, 2*orthoP, 0]
            grpAn.add(proStack8)
            slotlist[i] = 1
            #print('proStack8 1[',i+1,']', proStack8.position)
        if 14 == atomicNumber :
            buildProtonStack(2,i)
            proStack8.rotateX(piG/2)
            proStack8.position = [0,pP1pP2/2+2*orthoP, 0]
            grpAn.add(proStack8)
            slotlist[i] = 2
            #print('proStack8 2[',i+1,']', proStack8.position)
        if 15 == atomicNumber :
            buildProtonStack(1,i)
            proStack8.rotateX(piG/2)
            proStack8.position = [0, 2*orthoP, 0]
            grpAn.add(proStack8)
            slotlist[i] = 1
            #print('proStack8 1[',i+1,']', proStack8.position)
        if 16 <= atomicNumber < 36:
            buildProtonStack(2,i)
            proStack8.rotateX(piG/2)
            proStack8.position = [0,pP1pP2/2+2*orthoP, 0]
            grpAn.add(proStack8)
            slotlist[i] = 2
            #print('proStack8 2[',i+1,']', proStack8.position)
        if 36 <= atomicNumber < 54:
            buildProtonStack(4,i)
            proStack8.rotateX(piG/2)
            proStack8.position = [0,p1p2/2+pP1pP2+2*orthoP,0]
            grpAn.add(proStack8)
            slotlist[i] = 4
            #print('proStack8 4[',i+1,']', proStack8.position)
        if 54 <= atomicNumber < 57 :
            buildProtonStack(6,i)
            proStack8.rotateX(piG/2)
            proStack8.position = [0,p2p3+3*pP1pP2/2+2*orthoP, 0]
            grpAn.add(proStack8)
            slotlist[i] = 6
            #print('proStack8 6[',i+1,']', proStack8.position)
        if 57 <= atomicNumber < 71:
            buildProtonStack(5,i)
            proStack8.rotateX(piG/2)
            proStack8.position = [0,p2p3+pP1pP2+2*orthoP,0]
            grpAn.add(proStack8)
            slotlist[i] = 5
            #print('proStack8 5[',i+1,']', proStack8.position)
        if 71 <= atomicNumber :
            buildProtonStack(6,i)
            proStack8.rotateX(piG/2)
            proStack8.position = [0,p2p3+3*pP1pP2/2+2*orthoP, 0]
            grpAn.add(proStack8)
            slotlist[i] = 6
            #print('proStack8 6[',i+1,']', proStack8.position)

    if i == 8: # Slot 9
        proStack9 = Group()
        if 13 == atomicNumber :
            buildProtonStack(1,i)
            proStack9.rotateX(piG/2)
            proStack9.position = [0, -2*orthoP, 0]
            grpAn.add(proStack9)
            slotlist[i] = 1
            #print('proStack9 1[',i+1,']', proStack9.position)
            maxZNeg = -pP1pP2/2-4*orthoP
            maxZPos = 3*pP1pP2/2+6*orthoP
        if 14 == atomicNumber :
            buildProtonStack(2,i)
            proStack9.rotateX(piG/2)
            proStack9.position = [0,-pP1pP2/2-2*orthoP, 0]
            grpAn.add(proStack9)
            slotlist[i] = 2
            #print('proStack9 2[',i+1,']', proStack9.position)
        if 15 == atomicNumber :
            buildProtonStack(1,i)
            proStack9.rotateX(piG/2)
            proStack9.position = [0, -2*orthoP, 0]
            grpAn.add(proStack9)
            slotlist[i] = 1
            #print('proStack9 1[',i+1,']', proStack9.position)
        if 16 <= atomicNumber < 36:
            buildProtonStack(2,i)
            proStack9.rotateX(piG/2)
            proStack9.position = [0,-pP1pP2/2-2*orthoP, 0]
            grpAn.add(proStack9)
            slotlist[i] = 2
            #print('proStack9 2[',i+1,']', proStack9.position)
        if 36 <= atomicNumber < 54:
            buildProtonStack(4,i)
            proStack9.rotateX(piG/2)
            proStack9.position = [0,-2*orthoP-p1p2/2-pP1pP2,0]
            grpAn.add(proStack9)
            slotlist[i] = 4
            #print('proStack9 4[',i+1,']', proStack9.position)
        if 54 <= atomicNumber < 57:
            buildProtonStack(6,i)
            proStack9.rotateX(piG/2)
            proStack9.position = [0,-p2p3-3*pP1pP2/2-2*orthoP, 0]
            grpAn.add(proStack9)
            slotlist[i] = 6
            print('proStack9 6[',i+1,']', proStack9.position)
        if 57 <= atomicNumber < 71:
            buildProtonStack(5,i)
            proStack9.rotateX(piG/2)
            proStack9.position = [0,-p2p3-pP1pP2-2*orthoP,0]
            grpAn.add(proStack9)
            slotlist[i] = 5
            #print('proStack9 5[',i+1,']', proStack9.position)
        if 71 <= atomicNumber :
            buildProtonStack(6,i)
            proStack9.rotateX(piG/2)
            proStack9.position = [0,-p2p3-3*pP1pP2/2-2*orthoP, 0]
            grpAn.add(proStack9)
            slotlist[i] = 6
            #print('proStack9 6[',i+1,']', proStack9.position)

    if i == 9: # Slot 10
        proStack10 = Group()
        if 14 == atomicNumber:
            buildProtonStack(2,i)
            proStack10.rotateX(-piG/2)
            proStack10.rotateY(piG/2)
            proStack10.position = [pP1pP2/2+2*orthoP, 0, 0]
            grpAn.add(proStack10)
            slotlist[i] = 2
            #print('proStack10 2[',i+1,']', proStack10.position)
        if 15 == atomicNumber :
            buildProtonStack(2,i)
            proStack10.rotateX(-piG/2)
            proStack10.rotateY(piG/2)
            proStack10.position = [2*orthoP, 0, 0]
            grpAn.add(proStack10)
            slotlist[i] = 2
            #print('proStack10 2[',i+1,']', proStack10.position)
        if 16 == atomicNumber:
            buildProtonStack(2,i)
            proStack10.rotateX(-piG/2)
            proStack10.rotateY(piG/2)
            proStack10.position = [pP1pP2/2+2*orthoP, 0, 0]
            grpAn.add(proStack10)
            slotlist[i] = 2
            #print('proStack10 2[',i+1,']', proStack10.position)
        if 18 <= atomicNumber < 21 :
            buildProtonStack(2,i)
            proStack10.rotateX(-piG/2)
            proStack10.rotateY(piG/2)
            proStack10.position = [pP1pP2/2+2*orthoP, 0, 0]
            grpAn.add(proStack10)
            slotlist[i] = 2
            #print('proStack10 2[',i+1,']', proStack10.position)
        if 21 == atomicNumber :
            buildProtonStack(2,i)
            neutronGi.position = [0, 0, pP1pP2/2 + p1n2]
            proStack10.add(neutronGi)
            proStack10.rotateX(-piG/2)
            proStack10.rotateY(piG/2)
            proStack10.position = [pP1pP2/2+2*orthoP, 0, 0]
            grpAn.add(proStack10)
            slotlist[i] = 2
            #print('proStack10 2 w neutron[',i+1,']', proStack10.position)
        if 22 <= atomicNumber < 36 :
            buildProtonStack(2,i)
            proStack10.rotateX(-piG/2)
            proStack10.rotateY(piG/2)
            proStack10.position = [pP1pP2/2+2*orthoP, 0, 0]
            grpAn.add(proStack10)
            slotlist[i] = 2
            #print('proStack10 2[',i+1,']', proStack10.position)
        if 36 <= atomicNumber < 54:
            buildProtonStack(4,i)
            proStack10.rotateY(piG/2)
            proStack10.position = [2*orthoP+p1p2/2+pP1pP2,0,0]
            grpAn.add(proStack10)
            slotlist[i] = 4
            #print('proStack10 4[',i+1,']', proStack10.position)
        if 54 <= atomicNumber < 57 :
            buildProtonStack(6,i)
            proStack10.rotateY(piG/2)
            proStack10.position = [p2p3+3*pP1pP2/2+2*orthoP,0,0]
            grpAn.add(proStack10)
            slotlist[i] = 6
            #print('proStack10 6[',i+1,']', proStack10.position)
        if 57 <= atomicNumber < 71:
            buildProtonStack(5,i)
            proStack10.rotateY(piG/2)
            proStack10.position = [p2p3+pP1pP2+2*orthoP,0,0]
            grpAn.add(proStack10)
            slotlist[i] = 5
            #print('proStack10 5[',i+1,']', proStack10.position)
        if 71 <= atomicNumber :
            buildProtonStack(6,i)
            proStack10.rotateY(piG/2)
            proStack10.position = [p2p3+3*pP1pP2/2+2*orthoP,0,0]
            grpAn.add(proStack10)
            slotlist[i] = 6
            #print('proStack10 6[',i+1,']', proStack10.position)

    if i == 10: # Slot 11
        proStack11 = Group()
        if 14 == atomicNumber: # Silicon, the carousel level
            buildProtonStack(2,i)
            proStack11.rotateX(-piG/2)
            proStack11.rotateY(piG/2)
            proStack11.position = [-pP1pP2/2-2*orthoP, 0, 0]
            grpAn.add(proStack11)
            slotlist[i] = 2
            #print('proStack11 2[',i+1,']', proStack11.position)
        if 15 == atomicNumber:
            buildProtonStack(2,i)
            proStack11.rotateX(-piG/2)
            proStack11.rotateY(piG/2)
            proStack11.position = [-2*orthoP, 0, 0]
            grpAn.add(proStack11)
            slotlist[i] = 2
            #print('proStack11 2[',i+1,']', proStack11.position)
        if 16 == atomicNumber:
            buildProtonStack(2,i)
            proStack11.rotateX(-piG/2)
            proStack11.rotateY(piG/2)
            proStack11.position = [-pP1pP2/2-2*orthoP, 0, 0]
            grpAn.add(proStack11)
            slotlist[i] = 2
            #print('proStack11 2[',i+1,']', proStack11.position)
        if 18 <= atomicNumber < 21 :
            buildProtonStack(2,i)
            proStack11.rotateX(-piG/2)
            proStack11.rotateY(piG/2)
            proStack11.position = [-pP1pP2/2-2*orthoP, 0, 0]
            grpAn.add(proStack11)
            slotlist[i] = 2
            #print('proStack11 2[',i+1,']', proStack11.position)
        if 21 == atomicNumber :
            buildProtonStack(2,i)
            neutronGi.position = [0, 0, -pP1pP2/2 - p1n2]
            proStack11.add(neutronGi)
            proStack11.rotateX(-piG/2)
            proStack11.rotateY(piG/2)
            proStack11.position = [-pP1pP2/2-2*orthoP, 0, 0]
            grpAn.add(proStack11)
            slotlist[i] = 2
            #print('proStack11 2 w neutron[',i+1,']', proStack11.position)
        if 22 <= atomicNumber < 36 :
            buildProtonStack(2,i)
            proStack11.rotateX(-piG/2)
            proStack11.rotateY(piG/2)
            proStack11.position = [-pP1pP2/2-2*orthoP, 0, 0]
            grpAn.add(proStack11)
            slotlist[i] = 2
            #print('proStack11 2[',i+1,']', proStack11.position)
        if 36 <= atomicNumber < 54:
            buildProtonStack(4,i)
            proStack11.rotateY(piG/2)
            proStack11.position = [-2*orthoP-p1p2/2-pP1pP2,0,0]
            grpAn.add(proStack11)
            slotlist[i] = 4
            #print('proStack11 4[',i+1,']', proStack11.position)
        if 54 <= atomicNumber < 57:
            buildProtonStack(6,i)
            proStack11.rotateY(piG/2)
            proStack11.position = [-p2p3-3*pP1pP2/2-2*orthoP,0,0]
            grpAn.add(proStack11)
            slotlist[i] = 6
            #print('proStack11 6[',i+1,']', proStack11.position)
        if 57 <= atomicNumber < 71:
            buildProtonStack(6,i)
            proStack11.rotateY(piG/2)
            proStack11.position = [-p2p3-3*pP1pP2/2-2*orthoP,0,0]
            grpAn.add(proStack11)
            slotlist[i] = 6
            #print('proStack11 6[',i+1,']', proStack11.position)
        if 71 <= atomicNumber :
            buildProtonStack(6,i)
            proStack11.rotateY(piG/2)
            proStack11.position = [-p2p3-3*pP1pP2/2-2*orthoP,0,0]
            grpAn.add(proStack11)
            slotlist[i] = 6
            #print('proStack11 6[',i+1,']', proStack11.position)

    if i == 11: # Slot 12
        proStack12 = Group()
        if 21 <= atomicNumber < 27:
            buildProtonStack(1,i)
            neutronGi.position = [0,0,-p1n2]
            proStack12.add(neutronGi)
            proStack12.rotateZ(piG/2)
            proStack12.position = [0, pP1pP2 + 4*orthoP, 0]
            grpAn.add(proStack12)
            slotlist[i] = 1
            #print('proStack12 w neutron 1[',i+1,']', proStack12.position)
        if 27 == atomicNumber:
            neutronGi.position = [0,0,0]
            proStack12.add(neutronGi)
            proStack12.position = [0,p1n2+pP1pP2+2*orthoP,0]
            grpAn.add( proStack12 )##############################
            print('proStack12 neutron[',i+1,']', proStack12.position)
        if 28 == atomicNumber :
            buildProtonStack(1,i)
            neutronGi.position = [0,0,-p1n2]
            proStack12.add(neutronGi)
            proStack12.rotateZ(piG/2)
            proStack12.position = [0, pP1pP2+4*orthoP,0]
            grpAn.add(proStack12)##############################
            print('proStack12 w neutron 1[',i+1,']', proStack12.position)
        if 29 <= atomicNumber < 36 :
            buildProtonStack(2,i)
            proStack12.rotateZ(piG/2)
            proStack12.position = [0,pP1pP2+4*orthoP,0]
            grpAn.add(proStack12)
            slotlist[i] = 2
            #print('proStack12 2[',i+1,']', proStack12.position)
        if 42 <= atomicNumber < 47:
            buildProtonStack(1,i)
            proStack12.position = [0,4*pP1pP2/2+p1p2+4*orthoP,0]
            grpAn.add(proStack12)
            slotlist[i] = 1
            #print('proStack12 1[',i+1,']', proStack12.position)
        if 47 <= atomicNumber < 54 :
            buildProtonStack(2,i)
            proStack12.position = [0,4*pP1pP2/2+p1p2+4*orthoP,0]
            grpAn.add(proStack12)
            slotlist[i] = 2
            #print('proStack12 2[',i+1,']', proStack12.position)
        if 57 <= atomicNumber < 61:
            buildProtonStack(1,i)
            proStack12.position = [0,2*p2p3+2*pP1pP2+4*orthoP, 0]
            grpAn.add(proStack12)
            slotlist[i] = 1
            #print('proStack12 1[',i+1,']', proStack12.position)
        if 61 <= atomicNumber < 67:
            buildProtonStack(2,i)
            proStack12.position = [0, 2*p2p3+2*pP1pP2+4*orthoP, 0]
            grpAn.add(proStack12)
            slotlist[i] = 2
            #print('proStack12 2[',i+1,']', proStack12.position)
        if 67 == atomicNumber :
            buildProtonStack(3,i)
            proStack12.position = [0, 2*p2p3+2*pP1pP2+4*orthoP, 0]
            grpAn.add(proStack12)
            slotlist[i] = 3
            #print('proStack12 3[',i+1,']', proStack12.position)
        if 68 <= atomicNumber < 71 :
            buildProtonStack(4,i)
            proStack12.position = [0, 2*p2p3+2*pP1pP2+4*orthoP, 0]
            grpAn.add(proStack12)
            slotlist[i] = 4
            #print('proStack12 4[',i+1,']', proStack12.position)
        if 71 == atomicNumber :
            buildProtonStack(2,i)
            proStack12.position = [0, 2*p2p3+3*pP1pP2+4*orthoP, 0]
            grpAn.add(proStack12)
            slotlist[i] = 2
            #print('proStack12 2[',i+1,']', proStack12.position)
        if 72 <= atomicNumber < 79:
            buildProtonStack(3,i)
            proStack12.position = [0, 2*p2p3+3*pP1pP2+4*orthoP, 0]
            grpAn.add(proStack12)
            slotlist[i] = 3
            #print('proStack12 3[',i+1,']', proStack12.position)
        if 79 <= atomicNumber < 82 :
            buildProtonStack(4,i)
            proStack12.position = [0, 2*p2p3+3*pP1pP2+4*orthoP, 0]
            grpAn.add(proStack12)
            slotlist[i] = 4
            #print('proStack12 4[',i+1,']', proStack12.position)
        if 82 <= atomicNumber < 86:
            buildProtonStack(3,i)
            proStack12.position = [0, 2*p2p3+3*pP1pP2+4*orthoP, 0]
            grpAn.add(proStack12)
            slotlist[i] = 3
            #print('proStack12 3[',i+1,']', proStack12.position)
        if 86 <= atomicNumber :
            buildProtonStack(5,i)
            proStack12.position = [0, 2*p2p3+3*pP1pP2+4*orthoP, 0]
            grpAn.add(proStack12)
            slotlist[i] = 5
            #print('proStack12 5[',i+1,']', proStack12.position)

    if i == 12: # Slot 13
        proStack13 = Group()
        if 21 <= atomicNumber < 29:
            buildProtonStack(1,i)
            neutronGi.position = [0,0,-p1n2]
            proStack13.add(neutronGi)
            proStack13.rotateZ(piG/2)
            proStack13.position = [0, -pP1pP2 - 4*orthoP, 0]
            grpAn.add(proStack13)
            slotlist[i] = 1
            #print('proStack13 w neutron 1[',i+1,']', proStack13.position)
        if 29 <= atomicNumber < 36:
            buildProtonStack(2,i)
            proStack13.rotateZ(piG/2)
            proStack13.position = [0,-pP1pP2-4*orthoP,0]
            grpAn.add(proStack13)
            slotlist[i] = 2
            #print('proStack13 2[',i+1,']', proStack13.position)
        if 41 <= atomicNumber < 47:
            buildProtonStack(1,i)
            proStack13.rotateZ(piG/2)
            proStack13.position = [0,-4*pP1pP2/2-p1p2-4*orthoP,0]
            grpAn.add(proStack13)
            slotlist[i] = 1
            #print('proStack13 1[',i+1,']', proStack13.position)
        if 47 <= atomicNumber < 54:
            buildProtonStack(2,i)
            proStack13.position = [0,-4*pP1pP2/2-p1p2-4*orthoP,0]
            grpAn.add(proStack13)
            slotlist[i] = 2
            #print('proStack13 2[',i+1,']', proStack13.position)
        if 57 <= atomicNumber < 61:
            buildProtonStack(1,i)
            proStack13.rotateZ(piG/2)
            proStack13.position = [0, -2*p2p3-2*pP1pP2-4*orthoP, 0]
            grpAn.add(proStack13)
            slotlist[i] = 1
            #print('proStack13 1[',i+1,']', proStack13.position)
        if 61 <= atomicNumber < 67:
            buildProtonStack(2,i)
            proStack13.position = [0, -2*p2p3-2*pP1pP2-4*orthoP, 0]
            grpAn.add(proStack13)
            slotlist[i] = 2
            #print('proStack13 2[',i+1,']', proStack13.position)
        if 67 <= atomicNumber < 68:
            buildProtonStack(3,i)
            proStack13.position = [0, -2*p2p3-2*pP1pP2-4*orthoP, 0]
            grpAn.add(proStack13)
            slotlist[i] = 3
            #print('proStack13 3[',i+1,']', proStack13.position)
        if 68 <= atomicNumber < 71 :
            buildProtonStack(4,i)
            proStack13.position = [0, -2*p2p3-2*pP1pP2-4*orthoP, 0]
            grpAn.add(proStack13)
            slotlist[i] = 4
            #print('proStack13 4[',i+1,']', proStack13.position)
        if 71 <= atomicNumber < 79:
            buildProtonStack(3,i)
            proStack13.position = [0, -2*p2p3-3*pP1pP2-4*orthoP, 0]
            grpAn.add(proStack13)
            slotlist[i] = 3
            #print('proStack13 3[',i+1,']', proStack13.position)
        if 79 <= atomicNumber < 82 :
            buildProtonStack(4,i)
            proStack13.position = [0, -2*p2p3-3*pP1pP2-4*orthoP, 0]
            grpAn.add(proStack13)
            slotlist[i] = 4
            #print('proStack13 4[',i+1,']', proStack13.position)
        if 82 <= atomicNumber < 86:
            buildProtonStack(3,i)
            proStack13.position = [0, -2*p2p3-3*pP1pP2-4*orthoP, 0]
            grpAn.add(proStack13)
            slotlist[i] = 3
            #print('proStack13 3[',i+1,']', proStack13.position)
        if 86 <= atomicNumber :
            buildProtonStack(5,i)
            proStack13.position = [0, -2*p2p3-3*pP1pP2-4*orthoP, 0]
            grpAn.add(proStack13)
            slotlist[i] = 5
            #print('proStack13 5[',i+1,']', proStack13.position)

    if i == 13: # Slot 14
        proStack14 = Group()
        if 22 <= atomicNumber < 27:
            buildProtonStack(1,i)
            neutronGi.position = [0,0,-p1n2]
            proStack14.add(neutronGi)
            proStack14.rotateZ(-piG/2)
            proStack14.position = [4*orthoP+pP1pP2,0,0]
            grpAn.add(proStack14)
            slotlist[i] = 1
            #print('proStack14 1 w neutron[',i+1,']', proStack14.position)
        if 27 <= atomicNumber < 36:
            buildProtonStack(2,i)
            proStack14.rotateZ(-piG/2)
            proStack14.position = [4*orthoP+pP1pP2,0,0]
            grpAn.add(proStack14)
            slotlist[i] = 2
            #print('proStack14 2[',i+1,']', proStack14.position)
        if 39 <= atomicNumber < 48:
            buildProtonStack(1,i)
            proStack14.position = [4*pP1pP2/2+p1p2+4*orthoP,0,0]
            grpAn.add(proStack14)
            slotlist[i] = 1
            #print('proStack14 1[',i+1,']', proStack14.position)
        if 48 <= atomicNumber < 54:
            buildProtonStack(2,i)
            proStack14.position = [4*pP1pP2/2+p1p2+4*orthoP,0,0]
            grpAn.add(proStack14)
            slotlist[i] = 2
            print('proStack14 2[',i+1,']', proStack14.position)
        if 57 <= atomicNumber < 63:
            buildProtonStack(2,i)
            proStack14.position = [2*p2p3+2*pP1pP2+4*orthoP,0,0]
            grpAn.add(proStack14)
            slotlist[i] = 2
            #print('proStack14 2[',i+1,']', proStack14.position)
        if 63 <= atomicNumber < 71:
            buildProtonStack(3,i)
            proStack14.position = [2*p2p3+2*pP1pP2+4*orthoP,0,0]
            grpAn.add(proStack14)
            slotlist[i] = 3
            #print('proStack14 3[',i+1,']', proStack14.position)
        if 71 <= atomicNumber < 74:
            buildProtonStack(2,i)
            proStack14.position = [2*p2p3+3*pP1pP2+4*orthoP,0,0]
            grpAn.add(proStack14)
            slotlist[i] = 2
            #print('proStack14 2[',i+1,']', proStack14.position)
        if 74 <= atomicNumber < 80:
            buildProtonStack(3,i)
            proStack14.position = [2*p2p3+3*pP1pP2+4*orthoP,0,0]
            grpAn.add(proStack14)
            slotlist[i] = 3
            #print('proStack14 3[',i+1,']', proStack14.position)
        if 80 <= atomicNumber < 82:
            buildProtonStack(4,i)
            proStack14.position = [2*p2p3+3*pP1pP2+4*orthoP,0,0]
            grpAn.add(proStack14)
            slotlist[i] = 4
            #print('proStack14 4[',i+1,']', proStack14.position)
        if 82 <= atomicNumber :
            buildProtonStack(5,i)
            proStack14.position = [2*p2p3+3*pP1pP2+4*orthoP,0,0]
            grpAn.add(proStack14)
            slotlist[i] = 5
            #print('proStack14 5[',i+1,']', proStack14.position)

    if i == 14: # Slot 15
        proStack15 = Group()
        if 22 <= atomicNumber < 27 :
            buildProtonStack(1,i)
            neutronGi.position = [0,0,-p1n2]
            proStack15.add(neutronGi)
            proStack15.rotateZ(-piG/2)
            proStack15.position = [-4*orthoP-pP1pP2,0,0]
            grpAn.add(proStack15)
            slotlist[i] = 1
            #print('proStack15 1 w neutron[',i+1,']', proStack15.position)
        if 27 <= atomicNumber < 36 :
            buildProtonStack(2,i)
            proStack15.rotateZ(-piG/2)
            proStack15.position = [-4*orthoP-pP1pP2,0,0]
            grpAn.add(proStack15)
            slotlist[i] = 2
            #print('proStack15 2[',i+1,']', proStack15.position)
        if 39 <= atomicNumber < 48:
            buildProtonStack(1,i)
            proStack15.position = [-4*pP1pP2/2-p1p2-4*orthoP,0,0]
            grpAn.add(proStack15)
            slotlist[i] = 1
            #print('proStack15 1[',i+1,']', proStack15.position)
        if 48 <= atomicNumber < 54:
            buildProtonStack(2,i)
            proStack15.position = [-4*pP1pP2/2-p1p2-4*orthoP,0,0]
            grpAn.add(proStack15)
            slotlist[i] = 2
            #print('proStack15 2[',i+1,']', proStack15.position)
        if 57 <= atomicNumber < 63 :
            buildProtonStack(2,i)
            proStack15.position = [-2*p2p3-2*pP1pP2-4*orthoP,0,0]
            grpAn.add(proStack15)
            slotlist[i] = 2
            #print('proStack15 2[',i+1,']', proStack15.position)
        if 63 <= atomicNumber < 71 :
            buildProtonStack(3,i)
            proStack15.position = [-2*p2p3-2*pP1pP2-4*orthoP,0,0]
            grpAn.add(proStack15)
            slotlist[i] = 3
            #print('proStack15 3[',i+1,']', proStack15.position)
        if 71 == atomicNumber :
            buildProtonStack(3,i)
            proStack15.position = [-2*p2p3-3*pP1pP2-4*orthoP, 0, 0]
            grpAn.add(proStack15)
            slotlist[i] = 3
            #print('proStack15 3[',i+1,']', proStack15.position)
        if 72 <= atomicNumber < 74 :
            buildProtonStack(2,i)
            proStack15.position = [-2*p2p3-3*pP1pP2-4*orthoP,0,0]
            grpAn.add(proStack15)
            slotlist[i] = 2
            #print('proStack15 2[',i+1,']', proStack15.position)
        if 74 == atomicNumber :
            buildProtonStack(3,i)
            proStack15.position = [-2*p2p3-3*pP1pP2-4*orthoP,0,0]
            grpAn.add(proStack15)
            slotlist[i] = 3
            #print('proStack15 3[',i+1,']', proStack15.position)
        if 75 == atomicNumber :
            buildProtonStack(2,i)
            proStack15.position = [-2*p2p3-3*pP1pP2-4*orthoP,0,0]
            grpAn.add(proStack15)
            slotlist[i] = 2
            #print('proStack15 2[',i+1,']', proStack15.position)
        if 76 <= atomicNumber < 80 :
            buildProtonStack(3,i)
            proStack15.position = [-2*p2p3-3*pP1pP2-4*orthoP,0,0]
            grpAn.add(proStack15)
            slotlist[i] = 3
            #print('proStack15 3[',i+1,']', proStack15.position)
        if 80 <= atomicNumber < 82:
            buildProtonStack(4,i)
            proStack15.position = [-2*p2p3-3*pP1pP2-4*orthoP,0,0]
            grpAn.add(proStack15)
            slotlist[i] = 4
            #print('proStack15 4[',i+1,']', proStack15.position)
        if 82 <= atomicNumber :
            buildProtonStack(5,i)
            proStack15.position = [-2*p2p3-3*pP1pP2-4*orthoP,0,0]
            grpAn.add(proStack15)
            slotlist[i] = 5
            #print('proStack15 5[',i+1,']', proStack15.position)

    if i == 15: # Slot 16
        proStack16 = Group()
        if 27 <= atomicNumber < 30:
            proStack16.add(neutronGi)
            proStack16.position = [0,pP1pP2/2+p1n2,pP1pP2/2 + 2*orthoP]
            grpBn.add(proStack16)
            #print('proStack16 hook neutron[',i+1,']', proStack16.position)
        if 33 <= atomicNumber < 36:
            buildProtonStack(1,i)
            proStack16.position = [0, pP1pP2/2+2*orthoP, pP1pP2/2+2*orthoP]
            grpBn.add(proStack16)
            slotlist[i] = 1
            #print('proStack16 hook 1[',i+1,']', proStack16.position)
        if 46 <= atomicNumber < 48:
            buildProtonStack(1,i)
            proStack16.position = [0,p1p2/2+pP1pP2+2*orthoP,p1p2/2+pP1pP2+2*orthoP]
            grpBn.add(proStack16)
            slotlist[i] = 1
            #print('proStack16 hook 1[',i+1,']', proStack16.position)
        if 49 == atomicNumber:
            buildProtonStack(1,i)
            proStack16.position = [0,p1p2/2+pP1pP2+2*orthoP,p1p2/2+pP1pP2+2*orthoP]
            grpBn.add(proStack16)
            slotlist[i] = 1
            #print('proStack16 hook 1[',i+1,']', proStack16.position)
        if 53 == atomicNumber:
            buildProtonStack(2,i)
            proStack16.position = [0,p1p2/2+pP1pP2+2*orthoP,p1p2/2+pP1pP2+2*orthoP]
            grpBn.add(proStack16)
            slotlist[i] = 2
            #print('proStack16 hook 2[',i+1,']', proStack16.position)
        if 60 <= atomicNumber < 62:
            buildProtonStack(1,i)
            proStack16.position = [0,p2p3+pP1pP2+2*orthoP,p2p3+pP1pP2+2*orthoP]
            grpBn.add(proStack16)
            slotlist[i] = 1
            #print('proStack16 hook 1[',i+1,']', proStack16.position)
        if 64 <= atomicNumber < 66:
            buildProtonStack(2,i)
            proStack16.position = [0,p2p3+pP1pP2+2*orthoP,p2p3+pP1pP2+2*orthoP]
            grpBn.add(proStack16)
            slotlist[i] = 2
            #print('proStack16 hook 2[',i+1,']', proStack16.position)
        if 70 == atomicNumber :
            buildProtonStack(1,i)
            proStack16.position = [0,p2p3+pP1pP2+2*orthoP,p2p3+pP1pP2+2*orthoP]
            grpBn.add(proStack16)
            slotlist[i] = 1
            #print('proStack16 hook 1[',i+1,']', proStack16.position)
        if 72 <= atomicNumber < 74:
            buildProtonStack(1,i)
            proStack16.position = [0,p2p3+3*pP1pP2/2+2*orthoP,p2p3+3*pP1pP2/2+2*orthoP]
            grpBn.add(proStack16)
            slotlist[i] = 1
            #print('proStack16 hook 1[',i+1,']', proStack16.position)
        if 75 == atomicNumber :
            buildProtonStack(2,i)
            proStack16.position = [0,p2p3+3*pP1pP2/2+2*orthoP,p2p3+3*pP1pP2/2+2*orthoP]
            grpBn.add(proStack16)
            slotlist[i] = 2
            #print('proStack16 hook 2[',i+1,']', proStack16.position)
        if 77 <= atomicNumber < 80:
            buildProtonStack(2,i)
            proStack16.position = [0,p2p3+3*pP1pP2/2+2*orthoP,p2p3+3*pP1pP2/2+2*orthoP]
            grpBn.add(proStack16)
            slotlist[i] = 2
            #print('proStack16 hook 2[',i+1,']', proStack16.position)
        if 81 == atomicNumber :
            buildProtonStack(2,i)
            proStack16.position = [0,p2p3+3*pP1pP2/2+2*orthoP,p2p3+3*pP1pP2/2+2*orthoP]
            grpBn.add(proStack16)
            slotlist[i] = 2
            #print('proStack16 hook 2[',i+1,']', proStack16.position)
        if 82 == atomicNumber :
            buildProtonStack(1,i)
            proStack16.position = [0,p2p3+3*pP1pP2/2+2*orthoP,p2p3+3*pP1pP2/2+2*orthoP]
            grpBn.add(proStack16)
            slotlist[i] = 1
            #print('proStack16 hook 1[',i+1,']', proStack16.position)
        if 83 <= atomicNumber < 86 :
            buildProtonStack(2,i)
            proStack16.position = [0,p2p3+3*pP1pP2/2+2*orthoP,p2p3+3*pP1pP2/2+2*orthoP]
            grpBn.add(proStack16)
            slotlist[i] = 2
            #print('proStack16 hook 2[',i+1,']', proStack16.position)
        if 87 <= atomicNumber < 89 :
            buildProtonStack(1,i)
            proStack16.position = [0,p2p3+3*pP1pP2/2+2*orthoP,p2p3+3*pP1pP2/2+2*orthoP]
            grpBn.add(proStack16)
            slotlist[i] = 1
            #print('proStack16 hook 1[',i+1,']', proStack16.position)
        if 89 <= atomicNumber :
            buildProtonStack(2,i)
            proStack16.position = [0,p2p3+3*pP1pP2/2+2*orthoP,p2p3+3*pP1pP2/2+2*orthoP]
            grpBn.add(proStack16)
            slotlist[i] = 2
            #print('proStack16 hook 2[',i+1,']', proStack16.position)

    if i == 16: # Slot 17
        proStack17 = Group()
        if 26 <= atomicNumber < 30:
            proStack17.add(neutronGi)
            proStack17.position = [0,-pP1pP2/2-p1n2,-pP1pP2/2 - 2*orthoP]
            grpBn.add( proStack17 )
            #print('proStack17 hook neutron[',i+1,']', proStack17.position)
        if 34 <= atomicNumber < 36:
            buildProtonStack(1,i)
            proStack17.position = [0,-pP1pP2/2- 2*orthoP,-pP1pP2/2 - 2*orthoP]
            grpBn.add(proStack17)
            slotlist[i] = 1
            #print('proStack17 hook 1[',i+1,']', proStack17.position)
        if 45 <= atomicNumber < 48:
            buildProtonStack(1,i)
            proStack17.position = [0,-p1p2/2-pP1pP2-2*orthoP,-p1p2/2-pP1pP2-2*orthoP]
            grpBn.add(proStack17)
            slotlist[i] = 1
            #print('proStack17 hook 1[',i+1,']', proStack17.position)
        if 51 == atomicNumber:
            buildProtonStack(1,i)
            proStack17.position = [0,-p1p2/2-pP1pP2-2*orthoP,-p1p2/2-pP1pP2-2*orthoP]
            grpBn.add(proStack17)
            slotlist[i] = 1
            #print('proStack17 hook 1[',i+1,']', proStack17.position)
        if 53 == atomicNumber :
            buildProtonStack(1,i)
            proStack17.position = [0,-p1p2/2-pP1pP2-2*orthoP,-p1p2/2-pP1pP2-2*orthoP]
            grpBn.add(proStack17)
            slotlist[i] = 1
            #print('proStack17 hook 1[',i+1,']', proStack17.position)
        if 59 <= atomicNumber < 63 :
            buildProtonStack(1,i)
            proStack17.position = [0,-p2p3-pP1pP2-2*orthoP,-p2p3-pP1pP2-2*orthoP]
            grpBn.add(proStack17)
            slotlist[i] = 1
            #print('proStack17 hook 1[',i+1,']', proStack17.position)
        if 65 <= atomicNumber < 66 :
            buildProtonStack(1,i)
            proStack17.position = [0,-p2p3-pP1pP2-2*orthoP,-p2p3-pP1pP2-2*orthoP]
            grpBn.add(proStack17)
            slotlist[i] = 1
            #print('proStack17 hook 1[',i+1,']', proStack17.position)
        if 68 == atomicNumber :
            buildProtonStack(1,i)
            proStack17.position = [0,-p2p3-pP1pP2-2*orthoP,-p2p3-pP1pP2-2*orthoP]
            grpBn.add(proStack17)
            slotlist[i] = 1
            #print('proStack17 hook 1[',i+1,']', proStack17.position)
        if 69 == atomicNumber :
            buildProtonStack(2,i)
            proStack17.position = [0,-p2p3-pP1pP2-2*orthoP,-p2p3-pP1pP2-2*orthoP] 
            grpBn.add(proStack17)
            slotlist[i] = 2
            #print('proStack17 hook 2[',i+1,']', proStack17.position) 
        if 70 == atomicNumber :
            buildProtonStack(1,i)
            proStack17.position = [0,-p2p3-pP1pP2-2*orthoP,-p2p3-pP1pP2-2*orthoP]
            grpBn.add(proStack17)
            slotlist[i] = 1
            #print('proStack17 hook 1[',i+1,']', proStack17.position)
        if 72 == atomicNumber :
            buildProtonStack(1,i)
            proStack17.position = [0,-p2p3-3*pP1pP2/2-2*orthoP,-p2p3-3*pP1pP2/2-2*orthoP]
            grpBn.add(proStack17)
            slotlist[i] = 1
            #print('proStack17 hook 1[',i+1,']', proStack17.position)
        if 73 == atomicNumber :
            buildProtonStack(2,i)
            proStack17.position = [0,-p2p3-3*pP1pP2/2-2*orthoP,-p2p3-3*pP1pP2/2-2*orthoP]
            grpBn.add(proStack17)
            slotlist[i] = 2
            #print('proStack17 hook 2[',i+1,']', proStack17.position)
        if 74 == atomicNumber :
            buildProtonStack(1,i)
            proStack17.position = [0,-p2p3-3*pP1pP2/2-2*orthoP,-p2p3-3*pP1pP2/2-2*orthoP]
            grpBn.add(proStack17)
            slotlist[i] = 1
            #print('proStack17 hook 1[',i+1,']', proStack17.position)
        if 75 == atomicNumber :
            buildProtonStack(2,i)
            proStack17.position = [0,-p2p3-3*pP1pP2/2-2*orthoP,-p2p3-3*pP1pP2/2-2*orthoP]
            grpBn.add(proStack17)
            slotlist[i] = 2
            #print('proStack17 hook 2[',i+1,']', proStack17.position)
        if 77 <= atomicNumber < 80 :
            buildProtonStack(2,i)
            proStack17.position = [0,-p2p3-3*pP1pP2/2-2*orthoP,-p2p3-3*pP1pP2/2-2*orthoP]
            grpBn.add(proStack17)
            slotlist[i] = 2
            #print('proStack17 hook 2[',i+1,']', proStack17.position)
        if 81 <= atomicNumber < 84:
            buildProtonStack(1,i)
            proStack17.position = [0,-p2p3-3*pP1pP2/2-2*orthoP,-p2p3-3*pP1pP2/2-2*orthoP]
            grpBn.add(proStack17)
            slotlist[i] = 1
            #print('proStack17 hook 1[',i+1,']', proStack17.position)
        if 84 <= atomicNumber < 86 :
            buildProtonStack(2,i)
            proStack17.position = [0,-p2p3-3*pP1pP2/2-2*orthoP,-p2p3-3*pP1pP2/2-2*orthoP]
            grpBn.add(proStack17)
            slotlist[i] = 2
            #print('proStack17 hook 2[',i+1,']', proStack17.position)
        if 87 <= atomicNumber < 90 :
            buildProtonStack(1,i)
            proStack17.position = [0,-p2p3-3*pP1pP2/2-2*orthoP,-p2p3-3*pP1pP2/2-2*orthoP]
            grpBn.add(proStack17)
            slotlist[i] = 1
            #print('proStack17 hook 1[',i+1,']', proStack17.position)
        if 90 <= atomicNumber :
            buildProtonStack(2,i)
            proStack17.position = [0,-p2p3-3*pP1pP2/2-2*orthoP,-p2p3-3*pP1pP2/2-2*orthoP]
            grpBn.add(proStack17)
            slotlist[i] = 2
            #print('proStack17 hook 2[',i+1,']', proStack17.position)

    if i == 17: # Slot 18
        proStack18 = Group()
        if 25 == atomicNumber:
            buildProtonStack(1,i)
            proStack18.position = [0,-pP1pP2/2-2*orthoP, pP1pP2/2+2*orthoP]
            grpBn.add(proStack18)
            slotlist[i] = 1
            #print('proStack18 hook 1[',i+1,']', proStack18.position)
        if 26 <= atomicNumber < 29:
            proStack18.add(neutronGi)
            proStack18.position = [0,-pP1pP2/2-p1n2,pP1pP2/2 + 2*orthoP]
            grpBn.add( proStack18 )
            print('proStack18 hook neutron[',i+1,']', proStack18.position)
        if 29 <= atomicNumber < 30:
            neutronGi.position = [0,-n1n2/2,0]
            proStack18.add(neutronGi)
            neutronKi.position = [0,n1n2/2,0]
            proStack18.add(neutronKi)
            proStack18.rotateX(piG/2)
            proStack18.position = [0,-pP1pP2/2-p1n2,+pP1pP2/2+2*orthoP]
            grpBn.add( proStack18 )
            print('proStack18 w 2 hook neutrons[',i+1,']', proStack18.position)
        if 30 <= atomicNumber < 36:
            buildProtonStack(1,i)
            proStack18.position = [0,-pP1pP2/2-2*orthoP,pP1pP2/2+2*orthoP]
            grpBn.add(proStack18)
            slotlist[i] = 1
            #print('proStack18 hook 1[',i+1,']', proStack18.position)
        if 44 <= atomicNumber < 47:
            buildProtonStack(1,i)
            proStack18.position = [0,-p1p2/2-pP1pP2-2*orthoP,p1p2/2+pP1pP2+2*orthoP]
            grpBn.add(proStack18)
            slotlist[i] = 1
            #print('proStack18 hook 1[',i+1,']', proStack18.position)
        if 50 == atomicNumber:
            buildProtonStack(2,i)
            proStack18.position = [0,-p1p2/2-pP1pP2-2*orthoP,p1p2/2+pP1pP2+2*orthoP]
            grpBn.add(proStack18)
            slotlist[i] = 2
            #print('proStack18 hook 2[',i+1,']', proStack18.position)
        if 51 == atomicNumber:
            buildProtonStack(1,i)
            proStack18.position = [0,-p1p2/2-pP1pP2-2*orthoP,p1p2/2+pP1pP2+2*orthoP]
            grpBn.add(proStack18)
            slotlist[i] = 1
            #print('proStack18 hook 1[',i+1,']', proStack18.position)
        if 52 == atomicNumber :
            buildProtonStack(2,i)
            proStack18.position = [0,-p1p2/2-pP1pP2-2*orthoP,p1p2/2+pP1pP2+2*orthoP]
            grpBn.add(proStack18)
            slotlist[i] = 2
            #print('proStack18 hook 2[',i+1,']', proStack18.position)
        if 53 == atomicNumber :
            buildProtonStack(1,i)
            proStack18.position = [0,-p1p2/2-pP1pP2-2*orthoP,p1p2/2+pP1pP2+2*orthoP]
            grpBn.add(proStack18)
            slotlist[i] = 1
            #print('proStack18 hook 1[',i+1,']', proStack18.position)
        if 57 <= atomicNumber < 68 :
            buildProtonStack(2,i)
            proStack18.position = [0,-p2p3-pP1pP2-2*orthoP,p2p3+pP1pP2+2*orthoP]
            grpBn.add(proStack18)
            slotlist[i] = 2
            #print('proStack18 hook 2[',i+1,']', proStack18.position)
        if 70 == atomicNumber :
            buildProtonStack(1,i)
            proStack18.position = [0,-p2p3-pP1pP2-2*orthoP,p2p3+pP1pP2+2*orthoP]
            grpBn.add(proStack18)
            slotlist[i] = 1
            #print('proStack18 hook 1[',i+1,']', proStack18.position)
        if 71 == atomicNumber :
            buildProtonStack(1,i)
            proStack18.position = [0,-p2p3-3*pP1pP2/2-2*orthoP,p2p3+3*pP1pP2/2+2*orthoP]
            grpBn.add(proStack18)
            slotlist[i] = 1
            #print('proStack18 hook 1[',i+1,']', proStack18.position)
        if 76 == atomicNumber:
            buildProtonStack(2,i)
            proStack18.position = [0,-p2p3-pP1pP2-2*orthoP,p2p3+pP1pP2+2*orthoP]
            grpBn.add(proStack18)
            slotlist[i] = 2
            #print('proStack18 hook 2[',i+1,']', proStack18.position)
        if 77 == atomicNumber :
            buildProtonStack(1,i)
            proStack18.position = [0,-p2p3-3*pP1pP2/2-2*orthoP,p2p3+3*pP1pP2/2+2*orthoP]
            grpBn.add(proStack18)
            slotlist[i] = 1
            #print('proStack18 hook 1[',i+1,']', proStack18.position)
        if 80 <= atomicNumber < 81 :
            buildProtonStack(1,i)
            proStack18.position = [0,-p2p3-3*pP1pP2/2-2*orthoP,p2p3+3*pP1pP2/2+2*orthoP]
            grpBn.add(proStack18)
            slotlist[i] = 1
            #print('proStack18 hook 1[',i+1,']', proStack18.position)
        if 85 <= atomicNumber :
            buildProtonStack(1,i)
            proStack18.position = [0,-p2p3-3*pP1pP2/2-2*orthoP,p2p3+3*pP1pP2/2+2*orthoP]
            grpBn.add(proStack18)
            slotlist[i] = 1
            #print('proStack18 hook 1[',i+1,']', proStack18.position)

    if i == 18: # Slot 19
        proStack19 = Group()
        if 26 <= atomicNumber < 29:
            proStack19.add(neutronGi)
            proStack19.position = [0,pP1pP2/2+p1n2,-pP1pP2/2 - 2*orthoP]
            grpBn.add(proStack19)
            print('proStack19 hook neutron[',i+1,']', proStack19.position)
        if 29 <= atomicNumber < 30:
            neutronGi.position = [0,-n1n2/2,0]
            proStack19.add(neutronGi)
            neutronKi.position = [0,n1n2/2,0]
            proStack19.add(neutronKi)
            proStack19.rotateX(piG/2)
            proStack19.position = [0,pP1pP2/2+p1n2,-pP1pP2/2-2*orthoP]
            grpBn.add( proStack19 )
            print('proStack19 w 2 hook neutrons[',i+1,']', proStack19.position)
        if 30 == atomicNumber :
            buildProtonStack(1,i)
            proStack19.position = [0,pP1pP2/2+2*orthoP,-pP1pP2/2-2*orthoP]
            grpBn.add(proStack19)
            slotlist[i] = 1
            #print('proStack19 hook 1[',i+1,']', proStack19.position)
        if 32 <= atomicNumber < 35 : 
            buildProtonStack(1,i)
            proStack19.position = [0,pP1pP2/2+2*orthoP,-pP1pP2/2-2*orthoP]
            grpBn.add(proStack19)
            slotlist[i] = 1
            #print('proStack19 hook 1[',i+1,']', proStack19.position)
        if 35 == atomicNumber :
            buildProtonStack(2,i)
            proStack19.position = [0,pP1pP2/2+2*orthoP,-pP1pP2/2-2*orthoP]
            grpBn.add(proStack19)
            slotlist[i] = 2
            #print('proStack19 hook 2[',i+1,']', proStack19.position)
        if 43 <= atomicNumber < 47:
            buildProtonStack(1,i)
            proStack19.position = [0,p1p2/2+pP1pP2+2*orthoP,-p1p2/2-pP1pP2-2*orthoP]
            grpBn.add(proStack19)
            slotlist[i] = 1
            #print('proStack19 hook 1[',i+1,']', proStack19.position)
        if 50 == atomicNumber:
            buildProtonStack(2,i)
            proStack19.position = [0,p1p2/2+pP1pP2+2*orthoP,-p1p2/2-pP1pP2-2*orthoP]
            grpBn.add(proStack19)
            slotlist[i] = 2
            #print('proStack19 hook 2[',i+1,']', proStack19.position)
        if 51 == atomicNumber:
            buildProtonStack(1,i)
            proStack19.position = [0,p1p2/2+pP1pP2+2*orthoP,-p1p2/2-pP1pP2-2*orthoP]
            grpBn.add(proStack19)
            slotlist[i] = 1
            #print('proStack19 hook 1[',i+1,']', proStack19.position)
        if 52 == atomicNumber:
            buildProtonStack(2,i)
            proStack19.position = [0,p1p2/2+pP1pP2+2*orthoP,-p1p2/2-pP1pP2-2*orthoP]
            grpBn.add(proStack19)
            slotlist[i] = 2
            #print('proStack19 hook 2[',i+1,']', proStack19.position)
        if 53 == atomicNumber:
            buildProtonStack(1,i)
            proStack19.position = [0,p1p2/2+pP1pP2+2*orthoP,-p1p2/2-pP1pP2-2*orthoP]
            grpBn.add(proStack19)
            slotlist[i] = 1
            #print('proStack19 hook 1[',i+1,']', proStack19.position)
        if 57 == atomicNumber < 58:
            buildProtonStack(1,i)
            proStack19.position = [0,p2p3+pP1pP2+2*orthoP,-p2p3-pP1pP2-2*orthoP]
            grpBn.add(proStack19)
            slotlist[i] = 1
            #print('proStack19 hook 1[',i+1,']', proStack19.position)
        if 58 <= atomicNumber < 63:
            buildProtonStack(2,i)
            proStack19.position = [0,p2p3+pP1pP2+2*orthoP,-p2p3-pP1pP2-2*orthoP]
            grpBn.add(proStack19)
            slotlist[i] = 2
            #print('proStack19 hook 2[',i+1,']', proStack19.position)
        if 63 == atomicNumber :
            buildProtonStack(1,i)
            proStack19.position = [0,p2p3+pP1pP2+2*orthoP,-p2p3-pP1pP2-2*orthoP]
            grpBn.add(proStack19)
            slotlist[i] = 1
            #print('proStack19 hook 1[',i+1,']', proStack19.position)
        if 66 <= atomicNumber < 68 :
            buildProtonStack(2,i)
            proStack19.position = [0,p2p3+pP1pP2+2*orthoP,-p2p3-pP1pP2-2*orthoP]
            grpBn.add(proStack19)
            slotlist[i] = 2
            #print('proStack19 hook 2[',i+1,']', proStack19.position)
        if 68 <= atomicNumber < 71:
            buildProtonStack(1,i)
            proStack19.position = [0,p2p3+pP1pP2+2*orthoP,-p2p3-pP1pP2-2*orthoP]
            grpBn.add(proStack19)
            slotlist[i] = 1
            #print('proStack19 hook 1[',i+1,']', proStack19.position)
        if 71 == atomicNumber :
            buildProtonStack(1,i)
            proStack19.position = [0,p2p3+3*pP1pP2/2+2*orthoP,-p2p3-3*pP1pP2/2-2*orthoP]
            grpBn.add(proStack19)
            slotlist[i] = 1
            #print('proStack19 hook 1[',i+1,']', proStack19.position)
        if 74 == atomicNumber :
            buildProtonStack(1,i)
            proStack19.position = [0,p2p3+3*pP1pP2/2+2*orthoP,-p2p3-3*pP1pP2/2-2*orthoP]
            grpBn.add(proStack19)
            slotlist[i] = 1
            #print('proStack19 hook 1[',i+1,']', proStack19.position)
        if 76 == atomicNumber :
            buildProtonStack(2,i)
            proStack19.position = [0,p2p3+3*pP1pP2/2+2*orthoP,-p2p3-3*pP1pP2/2-2*orthoP]
            grpBn.add(proStack19)
            slotlist[i] = 2
            #print('proStack19 hook 2[',i+1,']', proStack19.position)
        if 80 <= atomicNumber < 81:
            buildProtonStack(1,i)
            proStack19.position = [0,p2p3+3*pP1pP2/2+2*orthoP,-p2p3-3*pP1pP2/2-2*orthoP]
            grpBn.add(proStack19)
            slotlist[i] = 1
            #print('proStack19 hook 1[',i+1,']', proStack19.position)
        if 86 == atomicNumber :
            buildProtonStack(1,i)
            proStack19.position = [0,p2p3+3*pP1pP2/2+2*orthoP,-p2p3-3*pP1pP2/2-2*orthoP]
            grpBn.add(proStack19)
            slotlist[i] = 1
            #print('proStack19 hook 1[',i+1,']', proStack19.position)
        if 88 <= atomicNumber :
            buildProtonStack(1,i)
            proStack19.position = [0,p2p3+3*pP1pP2/2+2*orthoP,-p2p3-3*pP1pP2/2-2*orthoP]
            grpBn.add(proStack19)
            slotlist[i] = 1
            #print('proStack19 hook 1[',i+1,']', proStack19.position)

grpAn.position = [0,0,(-maxZNeg + maxZPos)/2 - maxZPos]
grpBn.position = [0,0,(-maxZNeg + maxZPos)/2 - maxZPos]

#print(emissionType.value)
#print(number)
#print(slotlist)


proStack6 3[ 6 ] (0.0, 0.0, 3.4800000000000004)


In [97]:
# Create Slotlayout Diagram.

# Using 45 (currently non-interactive) buttons to create the atom's slotlayout.
# 19 in the atomic configuration, and 19, left(1,3,5,...,19) and right(2,4,6,...,18) columns  
# Each button includes: Tot number of slot protons, color, layout width and layout height
# as well as a total proton per slot color legend centered on the bottom row. 

# The 7 colors used for color-coding each slot's total proton counts, 0-6.
sclrs = ['white','lightgray','dodgerblue','pink','salmon','greenyellow','skyblue']     
# Below, slotlist, each slots' total proton count 
# Example
#slotlist = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]  # before the atom build
#slotlist = [2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  # post Neon build
sdlist = [slotlist[0],slotlist[1],slotlist[2],slotlist[3],slotlist[4],
         slotlist[5],slotlist[6],slotlist[7],slotlist[8],slotlist[9],
         slotlist[10],slotlist[11],slotlist[12],slotlist[13],slotlist[14],
         slotlist[15],slotlist[16],slotlist[17],slotlist[18]]
          
# The colors for each of the slots.
sclist = [sclrs[slotlist[0]],sclrs[slotlist[1]],sclrs[slotlist[2]],
          sclrs[slotlist[3]],sclrs[slotlist[4]],sclrs[slotlist[5]],
          sclrs[slotlist[6]],sclrs[slotlist[7]],sclrs[slotlist[8]],
          sclrs[slotlist[9]],sclrs[slotlist[10]],sclrs[slotlist[11]],
          sclrs[slotlist[12]],sclrs[slotlist[13]],sclrs[slotlist[14]],
          sclrs[slotlist[15]],sclrs[slotlist[16]],sclrs[slotlist[17]],
          sclrs[slotlist[18]]]

# Defining the slot buttons by adding the slot position, count and colors
s1 = Button(description='1 --- '+str(sdlist[0]), style=dict(button_color=sclist[0]))
s2 = Button(description='2 | '+str(sdlist[1]), style=dict(button_color=sclist[1]))
s3 = Button(description='3 | '+str(sdlist[2]), style=dict(button_color=sclist[2]))
s4 = Button(description='4 --- '+str(sdlist[3]), style=dict(button_color=sclist[3]))
s5 = Button(description='5 --- '+str(sdlist[4]), style=dict(button_color=sclist[4]))
s6 = Button(description='6 | '+str(sdlist[5]), style=dict(button_color=sclist[5]))
s7 = Button(description='7 | '+str(sdlist[6]), style=dict(button_color=sclist[6]))
s8 = Button(description='8 | '+str(sdlist[7]), style=dict(button_color=sclist[7]))
s9 = Button(description='9 | '+str(sdlist[8]), style=dict(button_color=sclist[8]))
s10 = Button(description='10 | '+str(sdlist[9]), style=dict(button_color=sclist[9]))
s11 = Button(description='11 | '+str(sdlist[10]), style=dict(button_color=sclist[10]))
s12 = Button(description='12 -- '+str(sdlist[11]), style=dict(button_color=sclist[11]))
s13 = Button(description='13 -- '+str(sdlist[12]), style=dict(button_color=sclist[12]))
s14 = Button(description='14 -- '+str(sdlist[13]), style=dict(button_color=sclist[13]))
s15 = Button(description='15 -- '+str(sdlist[14]), style=dict(button_color=sclist[14]))
s16 = Button(description='16 -- '+str(sdlist[15]), style=dict(button_color=sclist[15]))
s17 = Button(description='17 -- '+str(sdlist[16]), style=dict(button_color=sclist[16]))
s18 = Button(description='18 -- '+str(sdlist[17]), style=dict(button_color=sclist[17]))
s19 = Button(description='19 -- '+str(sdlist[18]), style=dict(button_color=sclist[18]))
# Slotlayout proton total color legend 
s20 = Button(description=str('0'), style=dict(button_color=sclrs[0]))
s21 = Button(description=str('1'), style=dict(button_color=sclrs[1]))
s22 = Button(description=str('2'), style=dict(button_color=sclrs[2]))
s23 = Button(description=str('3'), style=dict(button_color=sclrs[3]))
s24 = Button(description=str('4'), style=dict(button_color=sclrs[4]))
s25 = Button(description=str('5'), style=dict(button_color=sclrs[5]))
s26 = Button(description=str('6'), style=dict(button_color=sclrs[6]))
# Left and right slot columns, 
s27 = Button(description='1 --- '+str(sdlist[0]), style=dict(button_color=sclist[0]))
s28 = Button(description='2 | '+str(sdlist[1]), style=dict(button_color=sclist[1]))
s29 = Button(description='3 | '+str(sdlist[2]), style=dict(button_color=sclist[2]))
s30 = Button(description='4 --- '+str(sdlist[3]), style=dict(button_color=sclist[3]))
s31 = Button(description='5 --- '+str(sdlist[4]), style=dict(button_color=sclist[4]))
s32 = Button(description='6 | '+str(sdlist[5]), style=dict(button_color=sclist[5]))
s33 = Button(description='7 | '+str(sdlist[6]), style=dict(button_color=sclist[6]))
s34 = Button(description='8 | '+str(sdlist[7]), style=dict(button_color=sclist[7]))
s35= Button(description='9 | '+str(sdlist[8]), style=dict(button_color=sclist[8]))
s36 = Button(description='10 | '+str(sdlist[9]), style=dict(button_color=sclist[9]))
s37 = Button(description='11 | '+str(sdlist[10]), style=dict(button_color=sclist[10]))
s38 = Button(description='12 -- '+str(sdlist[11]), style=dict(button_color=sclist[11]))
s39 = Button(description='13 -- '+str(sdlist[12]), style=dict(button_color=sclist[12]))
s40 = Button(description='14 -- '+str(sdlist[13]), style=dict(button_color=sclist[13]))
s41 = Button(description='15 -- '+str(sdlist[14]), style=dict(button_color=sclist[14]))
s42 = Button(description='16 -- '+str(sdlist[15]), style=dict(button_color=sclist[15]))
s43 = Button(description='17 -- '+str(sdlist[16]), style=dict(button_color=sclist[16]))
s44 = Button(description='18 -- '+str(sdlist[17]), style=dict(button_color=sclist[17]))
s45 = Button(description='19 -- '+str(sdlist[18]), style=dict(button_color=sclist[18]))
# The slotlayout atom
s46 = Button(description=dlist[number-1]+' '+str(number), style=dict(button_color=sclrs[0]))


# Creating the list of buttons: 19,7,and 19
sbttns = [s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,
          s20,s21,s22,s23,s24,s25,s26,s27,s28,s29,s30,s31,s32,s33,s34,s35,s36,
          s37,s38,s39,s40,s41,s42,s43,s44,s45,s46]

# Add each button's border, layout width and height.
i=0
while i < 46 :
    sbttns[i].layout = widgets.Layout(border = '1px solid black') 
    sbttns[i].layout.width = '100%'
    sbttns[i].layout.height = '100%' 
    i += 1

# Layout the 46 (19 + 19 + 7 + 1) slotlayout buttons
sgrid = GridspecLayout(10, 9)

sgrid[1,4] = sbttns[5]
sgrid[2,4] = sbttns[3]
sgrid[2,5] = sbttns[10]
sgrid[2,6] = sbttns[14]
sgrid[3,3] = sbttns[17]
sgrid[3,4] = sbttns[1]
sgrid[3,5] = sbttns[15]
sgrid[4,2] = sbttns[12]
sgrid[4,3] = sbttns[8]
sgrid[4,4] = sbttns[0]
sgrid[4,5] = sbttns[7]
sgrid[4,6] = sbttns[11]
sgrid[5,3] = sbttns[16]
sgrid[5,4] = sbttns[2]
sgrid[5,5] = sbttns[18]
sgrid[6,2] = sbttns[13]
sgrid[6,3] = sbttns[9]
sgrid[6,4] = sbttns[4]
sgrid[7,4] = sbttns[6]
sgrid[9,1] = sbttns[19] # Color/num protons legend
sgrid[9,2] = sbttns[20] # along the bottom row
sgrid[9,3] = sbttns[21]
sgrid[9,4] = sbttns[22]
sgrid[9,5] = sbttns[23]
sgrid[9,6] = sbttns[24]
sgrid[9,7] = sbttns[25]
sgrid[0,0] = sbttns[26] # L/R Vertical slot columns
sgrid[0,8] = sbttns[27] 
sgrid[1,0] = sbttns[28]
sgrid[1,8] = sbttns[29]
sgrid[2,0] = sbttns[30]
sgrid[2,8] = sbttns[31]
sgrid[3,0] = sbttns[32]
sgrid[3,8] = sbttns[33]
sgrid[4,0] = sbttns[34]
sgrid[4,8] = sbttns[35]
sgrid[5,0] = sbttns[36]
sgrid[5,8] = sbttns[37]
sgrid[6,0] = sbttns[38]
sgrid[6,8] = sbttns[39]
sgrid[7,0] = sbttns[40]
sgrid[7,8] = sbttns[41]
sgrid[8,0] = sbttns[42]
sgrid[8,8] = sbttns[43]
sgrid[9,0] = sbttns[44]
sgrid[9,8] = sbttns[45] # Atomic symbpl and number

sgrid  # Display the atom's slotlayout   

GridspecLayout(children=(Button(description='6 | 3', layout=Layout(border='1px solid black', grid_area='widget…

**The Slot Layout Diagram** 
All atoms can be described by the number of protons (0-6) in each 
of up to 19 slot positions. The main vertical up/down column slots
(1-7) begins with Hydrogen - a single proton in the center slot1. 
Helium's slot1 contains two protons. 

Slot 1 is connected to the four front(10,14)/back(11,15) left(9,13)
/right(8,12), directions which spins as a single group called the 
Carousel. Or refer to the directions as +/-x, +/-y, and main vertical 
column +/-z.

Hook slot positions: 16,17,18, and 19, are joined to the main 
up/down column at slots, 2 and 3.  

The 1-19 sequence order number is indicated by the left side number 
in the atomic configuration in the center and in the left and right 
side vertical columns.

The proton count for a given slot is indicated by the right side 
number in the atomic configuration and side columns.

The vertical column sequence shown is top left, then top right, 
then down one on the left, then down one on the right, … , odd 
slot numbers on the left and even slots on the right.

The vertical, | or horizontal --- characters indicate the protons'
equatorial emission planes viewed edgewise, orthogonal to the 
proton's spin axis.

The seven center buttons in the bottom row contains the total 
protons per slot color legend.

The selected atom’s symbol and atomic number are in the bottom 
right corner.


In [98]:
if atomsLabel.value == True:
    scene = Scene(children=[grpAn, grpBn, camera, key_light, ambient_light,
        make_text(atomicLabel,
                  height=0.25,
                  position = [-1,1,number*0.025 + 1.5])]) # move the label +z as the atom grows
else:
    scene = Scene(children=[grpAn, grpBn, camera, key_light, ambient_light])
controller = OrbitControls(controlling=camera, screenSpacePanning=True)

In [99]:
renderer = Renderer(camera=camera, scene=scene, controls=[controller],
                    width=view_width, height=view_height)
scene = Scene()  # initialising the scene

In [100]:
# Defining the rotation widget controls for slots 1 - 19. 
spinC_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, caroSecs], values=[0, 2*piG])
spinC_clip = AnimationClip(tracks=[spinC_track])
spinC_action = AnimationAction(AnimationMixer(grpAn), spinC_clip, grpAn)

# The remaining widgets rotate the individual slots. 
spinB2_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, 6.28])
spinB2_clip = AnimationClip(tracks=[spinB2_track])
spinB2_action = AnimationAction(AnimationMixer(proStack2), spinB2_clip, proStack2)

spinB3_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, 6.28])
spinB3_clip = AnimationClip(tracks=[spinB3_track])
spinB3_action = AnimationAction(AnimationMixer(proStack3), spinB3_clip, proStack3)

spinB4_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, 6.28])
spinB4_clip = AnimationClip(tracks=[spinB4_track])
spinB4_action = AnimationAction(AnimationMixer(proStack4), spinB4_clip, proStack4)

spinB5_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, 6.28])
spinB5_clip = AnimationClip(tracks=[spinB5_track])
spinB5_action = AnimationAction(AnimationMixer(proStack5), spinB5_clip, proStack5)

spinB6_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, 6.28])
spinB6_clip = AnimationClip(tracks=[spinB6_track])
spinB6_action = AnimationAction(AnimationMixer(proStack6), spinB6_clip, proStack6)

spinB7_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, 6.28])
spinB7_clip = AnimationClip(tracks=[spinB7_track])
spinB7_action = AnimationAction(AnimationMixer(proStack7), spinB7_clip, proStack7)

spinB8_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, 6.28])
spinB8_clip = AnimationClip(tracks=[spinB8_track])
spinB8_action = AnimationAction(AnimationMixer(proStack8), spinB8_clip, proStack8)

spinB9_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, 6.28])
spinB9_clip = AnimationClip(tracks=[spinB9_track])
spinB9_action = AnimationAction(AnimationMixer(proStack9), spinB9_clip, proStack9)

spinB10_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, 6.28])
spinB10_clip = AnimationClip(tracks=[spinB10_track])
spinB10_action = AnimationAction(AnimationMixer(proStack10), spinB10_clip, proStack10)

spinB11_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, 6.28])
spinB11_clip = AnimationClip(tracks=[spinB11_track])
spinB11_action = AnimationAction(AnimationMixer(proStack11), spinB11_clip, proStack11)

spinB12_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, 6.28])
spinB12_clip = AnimationClip(tracks=[spinB12_track])
spinB12_action = AnimationAction(AnimationMixer(proStack12), spinB12_clip, proStack12)

spinB13_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, 6.28])
spinB13_clip = AnimationClip(tracks=[spinB13_track])
spinB13_action = AnimationAction(AnimationMixer(proStack13), spinB13_clip, proStack13)

spinB14_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, 6.28])
spinB14_clip = AnimationClip(tracks=[spinB14_track])
spinB14_action = AnimationAction(AnimationMixer(proStack14), spinB14_clip, proStack14)

spinB15_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, 6.28])
spinB15_clip = AnimationClip(tracks=[spinB15_track])
spinB15_action = AnimationAction(AnimationMixer(proStack15), spinB15_clip, proStack15)

spinB16_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, 6.28])
spinB16_clip = AnimationClip(tracks=[spinB16_track])
spinB16_action = AnimationAction(AnimationMixer(proStack16), spinB16_clip, proStack16)

spinB17_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, 6.28])
spinB17_clip = AnimationClip(tracks=[spinB17_track])
spinB17_action = AnimationAction(AnimationMixer(proStack17), spinB17_clip, proStack17)

spinB18_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, 6.28])
spinB18_clip = AnimationClip(tracks=[spinB18_track])
spinB18_action = AnimationAction(AnimationMixer(proStack18), spinB18_clip, proStack18)

spinB19_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, 6.28])
spinB19_clip = AnimationClip(tracks=[spinB19_track])
spinB19_action = AnimationAction(AnimationMixer(proStack19), spinB19_clip, proStack19)# Rotation controls for slot 1, which spins the atom's carousel-left/right, front/back slots.

# Layout the 19 rotation controls
rgrid = GridspecLayout(8, 7)

rgrid[1, 3] = spinB6_action
rgrid[2, 3] = spinB4_action
rgrid[2, 4] = spinB11_action
rgrid[2, 5] = spinB15_action
rgrid[3, 2] = spinB18_action
rgrid[3, 3] = spinB2_action
rgrid[3, 4] = spinB16_action
rgrid[4, 1] = spinB13_action
rgrid[4, 2] = spinB9_action
rgrid[4, 3] = spinC_action
rgrid[4, 4] = spinB8_action
rgrid[4, 5] = spinB12_action
rgrid[5, 2] = spinB17_action
rgrid[5, 3] = spinB3_action
rgrid[5, 4] = spinB19_action
rgrid[6, 1] = spinB14_action
rgrid[6, 2] = spinB10_action
rgrid[6, 3] = spinB5_action
rgrid[7, 3] = spinB7_action


**Carousel and Slot Rotation Controls.** The following two SlotLayout arrays of 19 
rotation controls spin each of the atoms' up to 19 occupied slots. The rotation 
controls are displayed twice in order to give the user a better view of the atomic 
spin changes in response to the rotation control changes. 

Only a few of the larger atoms have all 19 slots occupied. Slot1 spins the carousel. 
Each object within each slot spins about the spin axis through the slot center at 
the same rate, which is not correct, it still provides a good idea how the atom 
spins. Eventually, all protons, neutrons and electrons should spin at their own 
rates. If a slot is unoccupied, the corresponding slot rotation control has nothing 
to spin and will not do anything.

Try to remember to stop any active rotations before selecting another large atom 
or things may slow down drastically.

In [101]:
rgrid # Display the top set of rotation controls.

GridspecLayout(children=(AnimationAction(clip=AnimationClip(duration=2.0, tracks=(NumberKeyframeTrack(name='.r…

In [102]:
renderer # Display the selected atom.

Renderer(camera=PerspectiveCamera(aspect=1.5, position=(5.0, 0.0, 0.0), projectionMatrix=(1.4296712803397058, …

In [103]:
rgrid # Display the top set of rotation controls.

GridspecLayout(children=(AnimationAction(clip=AnimationClip(duration=2.0, tracks=(NumberKeyframeTrack(name='.r…


Charge Field physics and the Unified Field theory, (and much more) have been well developed and well described by Miles Mathis. See,

***THE GREATEST STANDING ERRORS IN PHYSICS AND MATHEMATICS***
<http://milesmathis.com/index.html>
[Miles Mathis science site Homepage](http://milesmathis.com/index.html).he reference's **SECTION 9: THE NUCLEUS** contains descriptions
and diagrams of charge channeling and charge recycling by the 
elements. A paper most relevant to Atom Builder is 
>**How to Build the Elements**. 
>Explaining the periodic table, with nuclear diagrams".  
<http://milesmathis.com/nuclear.pdf>
[How to Build the Elements](http://milesmathis.com/nuclear.pdf)
For Charge Field discussion visit the forum at "Miles Mathis' Charge Field - Portal" https://milesmathis.forumotion.com.

This particular project is described in the Miles Periodic Table with Standard Periodic Table reference thread, https://milesmathis.forumotion.com/t634p75-miles-periodic-table-with-standard-periodic-table-reference#6702.

This project was Cr6's idea. He's also responsible for following Miles Mathis' atomic model and creating the Slotlayout diagram this project heavily relies on.

There's at least one 10 year old charge field "Atom Viewer" out there, Nevyn's object oriented, way sophisticated javaScript code. I'm ready to try and figure it out.